In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm, Normalize, CenteredNorm, SymLogNorm
import matplotlib.gridspec as gridspec
from sklearn.linear_model import LinearRegression, LogisticRegression
import pandas as pd
import numpy as np
from scipy.stats import linregress, t, gaussian_kde, mannwhitneyu, hmean, combine_pvalues, fisher_exact, ttest_1samp
from pybedtools import BedTool, helpers
import statsmodels.api as sm
from tqdm import tqdm
import multiprocessing as mp
helpers.set_tempdir(
        '/home/jupyter/workspaces/neanderthalintrogressionlandscapeinafricanamericans/introgression_african_americans')

### Plot admixutre proportions

In [ ]:
def read_ibdmix_results(filename):
    ibdmix = pd.read_csv(filename, sep='\t', names=['chrom', 'start', 'end', "LOD", "IID", 
                                                    "pop", "super_pop"])
    ibdmix['length'] = (ibdmix.end - ibdmix.start)
    return ibdmix

def load_npz(fn):
    arr = np.load(fn, allow_pickle=True)
    arr = arr['arr_0']
    return arr

def sort_individuals(df, K):
    """
    Sort individuals by modal ancestry components and it's intensity
    :param df: pd.DataFrame, data frame containing ancestry porportions
    :param K: int, K used to run admixture
    :return: pd.DataFrame, sorted df
    """
    # get mean ancestry per component for each population
    mean_ancestry_pops = df.groupby('population').mean()
    # identify modal component for each population, i.e., the component with the highest ancestry proportion
    modal_clusters_pops = mean_ancestry_pops.idxmax(axis=1).sort_values()
    individual_orders = []
    # iterate over all k
    for i in df.columns[:-1]:
        # get all populations for which the current component is model
        pops = modal_clusters_pops[modal_clusters_pops == i].index.values.tolist()
        # sort the population by they mean ancestry proportions of the modal component
        pops_sorted = mean_ancestry_pops.loc[pops, i].sort_values(ascending=False).index.values
        inds = []
        # sort the individuals within a population by ancestry proportion
        for pop in pops_sorted:
            inds.extend(df.loc[df.population == pop, i].sort_values(ascending=False).index.values.tolist())
        individual_orders.extend(inds)

    df = df.loc[individual_orders, :]
    return df


def plot_admixture_proportions(df, K):
    """
    Create horizontal bar plot of ADMIXTURE results
    :param df: pd.DataFrame, sorted DataFrame with ancestry proportions
    :param K: int, K used for admixture run
    """
    colors = ["red", "green", "blue", 'orange']
    # pad colors
    while K > len(colors):
        colors.append((np.random.random(), np.random.random(), np.random.random()))
    fig, ax = plt.subplots(figsize=(2, 8))
    populations = df.population.values
    y_coords = [0]
    y_ticks = []
    y_labels = []
    prev_pop = populations[0]
    pop_coord_start = 0
    cumulative_padding = 0
    padding = 2
    for i, pop in enumerate(populations[1:], 1):
        if prev_pop != pop:
            # get ytick positions and labels
            y_ticks.append((pop_coord_start + i + cumulative_padding - 1) / 2)
            if prev_pop == 'AA':
                y_labels.append('Admixed')
            else:
                y_labels.append(prev_pop)
            prev_pop = pop
            # add white space between populations
            cumulative_padding += padding
            pop_coord_start = i + cumulative_padding
        # determine y coords --> add white space between populations
        y_coords.append(i + cumulative_padding)
    y_ticks.append((pop_coord_start + i + cumulative_padding) / 2)
    if prev_pop == 'AA':
        y_labels.append('Admixed')
    else:
        y_labels.append(prev_pop)
    prev_k = []
    # do plotting
    for i, anc in enumerate(df.columns[:-1]):
        if i > 0:
            ax.barh(y_coords, df.loc[:, anc], left=df.loc[:, prev_k].sum(axis=1), height=1, color=colors[i])
        else:
            ax.barh(y_coords, df.loc[:, anc], height=1, color=colors[i])
        prev_k.append(anc)
    # formatting
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_labels, fontsize=10)
    ax.set_xticks(np.arange(0, 1.2, 0.2))
    ax.invert_yaxis()
    ax.set_xlabel(f"K={K}", fontsize=12)
    ax.set_ylim([len(populations) + cumulative_padding - 0.5, -0.5])
    ax.set_xlim([0, 1])
    return fig, ax

def read_flare(fname_pattern):
    dfs = []
    for chrom in range(1, 23):
        phase_0 = pd.read_csv(fname_pattern.format(chrom=chrom, phase=0),
                             header=0, sep='\t')
        phase_0['phase'] = 0
        phase_1 = pd.read_csv(fname_pattern.format(chrom=chrom, phase=1),
                             header=0, sep='\t')
        phase_1['phase'] = 1
        dfs.append(phase_0)
        dfs.append(phase_1)
    df = pd.concat(dfs)
    df['length'] = df.end - df.start
    return df


def linear_regression(x, y, full_range=False):
    reg = linregress(x, y)
    if not full_range:
        x2 = np.linspace(x.min(), x.max(), 100)
    else:
        x2 = np.linspace(min([x.min(), y.min()]), 
                         max([x.max(), y.max()]), 100)
    y2 = x2 * reg.slope + reg.intercept
#     https://github.com/BMClab/BMC/blob/0c338ecf74798379d724a0f27f399c96b7aecbe3//notebooks/CurveFitting.ipynb
    # Confidence interval for the linear fit:
    zscore = t.ppf(0.975, x.shape[0] - 2)
    s_err = s_err = np.sqrt(np.sum((y - (x * reg.slope + reg.intercept))**2)/(x.shape[0] - 2))
    ci = zscore * s_err * np.sqrt(1 / x.shape[0] + (x2 - np.mean(x))**2/np.sum((x2 - np.mean(x))**2))
    # Prediction interval for the linear fit:
    pi = zscore * s_err * np.sqrt(1 + 1 / x.shape[0] + (x2 - np.mean(x))**2 / np.sum((x2 - np.mean(x))**2))
    print(f'Slope: {reg.slope} 95% CI: {reg.slope - reg.stderr * 1.96} - {reg.slope + reg.stderr * 1.96}')
    print(f'Intercept: {reg.intercept} 95% CI: {reg.intercept - reg.intercept_stderr * 1.96} - {reg.intercept + reg.intercept_stderr * 1.96}')
    
    return reg.slope, reg.intercept, reg.rvalue, reg.pvalue, x2, y2, ci, pi
    

def make_violinplot(amounts, position, ax, color, width=0.5, alpha=1):
    parts = ax.violinplot(amounts, showmedians=True, positions=[position], widths=width)
    for pc in parts['bodies']:
        pc.set_facecolor(color)
        pc.set_edgecolor('black')
        pc.set_linewidth(0.1)
        pc.set_alpha(alpha)
    parts['cmedians'].set_color('black')
    parts['cmins'].set_color('black')
    parts['cmaxes'].set_color('black')
    parts['cbars'].set_color('black')
    return ax

def compare_amounts_predicted_introgressed_sequence_by_populations(df, neanderthal, ax=None, fig=None):
    if ax is None or fig is None:
        fig, ax = plt.subplots(figsize=(12, 6))
    amounts = df.groupby(['super_pop', 'pop', 'IID']).sum().loc[:, 'length'] / 1e6
    super_pops = amounts.index.get_level_values(0).unique()
    n = 0
    xlabels = []
    for super_pop, color in zip(['EAS', 'EUR', 'AFR', 'AMR'], 
                                ['Greens_r', 'Blues_r', 'Reds_r', 'purple']):
        if super_pop in super_pops:
            c_amounts = amounts.loc[(super_pop, slice(None), slice(None))]
        else:
            continue
        populations = c_amounts.index.get_level_values(0).unique()
        if len(populations) > 1:
            cmap = plt.colormaps[color]
        else:
            cmap = color
        for i, pop in enumerate(populations, 1):
            try:
                ax = make_violinplot(c_amounts.loc[pop, :], n, ax, cmap(i * 30))
            except TypeError:
                ax = make_violinplot(c_amounts.loc[pop, :], n, ax, cmap)
            n += 1
            if pop == 'AA':
                xlabels.append('AOU-\nAdmixed')
            else:
                xlabels.append(pop)
    ax.set_xticks(np.arange(0, len(xlabels)))
    ax.set_xticklabels(xlabels, rotation=60)
    ax.set_ylabel(f'{neanderthal} introgressed\nper individual (Mb)')
    return fig, ax


def plot_haplotype_frequencies(df, column, chrom, ax, alpha, color):
    c_starts = df.loc[df.chrom == chrom, 'start'].values / 1e6
#     c_freqs = df.loc[df.chrom == chrom, column].values
    c_ends = df.loc[df.chrom == chrom, 'end'].values / 1e6
    ax.bar(c_starts, 1, c_ends - c_starts, align='edge', color=color, alpha=alpha)
    return ax

# def plot_introgression_landscape(df, selected, columns, colors, alpha=None, fig=None, ax=None):
#     while len(alpha) < len(columns):
#         alpha.append(1)
#     while len(colors) < len(columns):
#         colors.append((np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255)))
#     if fig is None or ax is None:
#         fig, ax = plt.subplots(22, 1, figsize=(16, 16), sharex=True)
#         max_freq = df.loc[:, columns].max().max() + 0.02
#     else:
#         max_freq = max([df.loc[:, columns].max().max(), ax[0].get_ylim()[1]])
#     for chrom, start, end in selected.loc[:, ['chrom', 'start', 'end']].values:
#         ax[chrom - 1].plot([start, end], [0, 0], c='black', zorder=1)
#     for column, color, a in zip(columns, colors, alpha):
#         for chrom in range(22):
#             ax = plot_haplotype_frequencies(df.loc[:, ['chrom', 'start', 
#                                                        'end', column]].rename(columns={column: 'freq'}), 
#                                             chrom, ax, a, color)

#             ax[chrom].set_yticks([max_freq / 2])
#             ax[chrom].set_yticklabels([chrom + 1])
#             ax[chrom].set_ylim([0, max_freq])
#             for spine in ax[chrom].spines.values():
#                 spine.set_visible(False)
#             if chrom != 21:
#                 ax[chrom].tick_params(axis='both', bottom=False, left=False)
#             else:
#                 ax[chrom].tick_params(axis='both', bottom=True, left=False)

#     ax[chrom].set_xlabel('Coordinates in Mb')

#     return fig, ax

#### Global ancestries

In [ ]:
K = 3
admixture_proportions = pd.read_csv(f'data/merged_datasets/ancestry_proportions-20.7.Q', sep='\t', 
                                    header=0, index_col=0)
# retain only admixed individuals
aa_individuals = pd.read_csv('data/AA_sample_ids.txt', names=['sample'])
admixture_proportions = admixture_proportions[np.isin(admixture_proportions.index.values, 
                                                      aa_individuals['sample'].values)]
admixture_proportions.drop('SelfReportedRaceEthnicity', inplace=True, axis=1)
# add population for downstream compatibility
admixture_proportions['population'] = 'AA'
admixture_proportions.rename(columns={'African': 'AFR', 'European': 'EUR', 'EastAsian': 'EAS'}, inplace=True)
# aggregate Asian ancestries to EAS
admixture_proportions.EAS += admixture_proportions.SouthAsian
admixture_proportions.EAS += admixture_proportions.Oceania
admixture_proportions.EAS += admixture_proportions.WestAsian
admixture_proportions.EAS += admixture_proportions.NativeAmerican
# drop aggregated ancestries
admixture_proportions.drop(['SouthAsian', 'Oceania', 'WestAsian', 'NativeAmerican'], axis=1, inplace=True)
# divide 100
admixture_proportions.loc[:, ['AFR', 'EUR', 'EAS']] /= 100
# sort
admixture_proportions.sort_values(['AFR', 'EUR', 'EAS'], inplace=True)
admixture_proportions.index = admixture_proportions.index.values.astype(str)
# plot
# fig, ax = plot_admixture_proportions(admixture_proportions, K)
# fig.savefig('visualizations/admixture.pdf', bbox_inches='tight', dpi=600)

#### Local ancestries

In [ ]:
chrom_sizes = pd.read_csv('data/reference/hg38.chrom.sizes.bed', sep='\t', names=['chrom', 'start', 'end'])
flare_global = pd.read_csv('results50kb_wo_PEGS/flare/african_american_and_ref_individuals_chr1.global.anc.gz',
                           sep='\t')
flare_global.set_index('SAMPLE', inplace=True)
flare_global.sort_index()
flare_global *= chrom_sizes.loc[chrom_sizes.chrom == 'chr1', 'end'].values[0] / chrom_sizes['end'].sum()
for chrom in range(2, 23):
    c_flare_global = pd.read_csv(f'results50kb_wo_PEGS/flare/african_american_and_ref_individuals_chr{chrom}.global.anc.gz', 
                                 sep='\t')
    c_flare_global.set_index('SAMPLE', inplace=True)
    c_flare_global.sort_index()
    c_flare_global *= (chrom_sizes.loc[chrom_sizes.chrom == f'chr{chrom}', 'end'].values[0] / 
                       chrom_sizes['end'].sum())
    flare_global += c_flare_global
    
flare_global.index = flare_global.index.values.astype(str)
# join
flare_global = admixture_proportions.join(flare_global, rsuffix='_flare')
fig, ax = plt.subplots(1, 3, figsize=(10,6))
plt.subplots_adjust(wspace=0.45)
(slope_eas, intercept_eas, rval_eas, 
 pval_eas, x_model_eas, y_model_eas, ci_eas, pi_eas) = linear_regression(flare_global.EAS, 
                                                                         flare_global.EAS_flare,
                                                                         full_range=True)
(slope_eur, intercept_eur, rval_eur, 
 pval_eur, x_model_eur, y_model_eur, ci_eur, pi_eur) = linear_regression(flare_global.EUR, 
                                                                         flare_global.EUR_flare,
                                                                         full_range=True)
(slope_afr, intercept_afr, rval_afr, 
 pval_afr, x_model_afr, y_model_afr, ci_afr, pi_afr) = linear_regression(flare_global.AFR, 
                                                                         flare_global.AFR_flare,
                                                                         full_range=True)
ax[2].scatter(flare_global.EAS, flare_global.EAS_flare, s=1, c='green')
ax[2].plot(x_model_eas, y_model_eas, ls='--', color='black')
# ax[2].fill_between(x_model_eas, y_model_eas + ci_eas, y_model_eas - ci_eas, color="black", alpha=0.6)
if pval_eas <= 1e-6:
    ax[2].annotate('m={:.2f}; P'.format(slope_eas) + r'$\leq$' + '{:.0e}; '.format(1e-6) + r'$r$' + 
                   '={:.2f}'.format(rval_eas ** 2), (1, 1), (0.05, 0.93), 
                   xycoords='axes fraction', fontsize=8)
else:
    ax[2].annotate('m={:.2f}; P={:.2e}; '.format(slope_eas, pval_eas) + r'$r$' + 
                   '={:.2f}'.format(rval_eas ** 2), (1, 1), (0.05, 0.93), 
                   xycoords='axes fraction', fontsize=8)
ax[1].scatter(flare_global.EUR, flare_global.EUR_flare, s=1, c='blue', alpha=0.7)
ax[1].plot(x_model_eur, y_model_eur, ls='--', color='black')
# ax[1].fill_between(x_model_eur, y_model_eur + ci_eur, y_model_eur - ci_eur, color="black", alpha=0.6)
if pval_eur <= 1e-6:
    ax[1].annotate('m={:.2f}; P'.format(slope_eur) + r'$\leq$' + '{:.0e}; '.format(1e-6) + r'$r$' + 
                   '={:.2f}'.format(rval_eur ** 2), (1, 1), (0.05, 0.93), 
                   xycoords='axes fraction', fontsize=8)
else:
    ax[1].annotate('m={:.2f}; P={:.2e}; '.format(slope_eur, pval_eur) + r'$r$' + 
                   '={:.2f}'.format(rval_eur ** 2), (1, 1), (0.05, 0.93), 
                   xycoords='axes fraction', fontsize=8)

ax[0].scatter(flare_global.AFR, flare_global.AFR_flare, s=1, c='red')
ax[0].plot(x_model_afr, y_model_afr, ls='--', color='black')
# ax[0].fill_between(x_model_afr, y_model_afr + ci_afr, y_model_afr - ci_afr, color="black", alpha=0.6)
if pval_afr <= 1e-6:
    ax[0].annotate('m={:.2f}; P'.format(slope_afr) + r'$\leq$' + '{:.0e}; '.format(1e-6) + r'$r$' + 
                   '={:.2f}'.format(rval_afr ** 2), (1, 1), (0.05, 0.93), 
                   xycoords='axes fraction', fontsize=8)
else:
    ax[0].annotate('a={:.2f}; P={:.2e}; '.format(slope_afr, pval_afr) + r'$r$' + 
                   '={:.2f}'.format(rval_afr ** 2), (1, 1), (0.05, 0.93), 
                   xycoords='axes fraction', fontsize=8)

ax[2].set_aspect('equal')
ax[1].set_aspect('equal')
ax[0].set_aspect('equal')
ax[2].set_xlim([0, 0.06])
ax[2].set_ylim([0, 0.06])
ax[1].set_xlim([0.05, 0.55])
ax[1].set_ylim([0.05, 0.55])
ax[0].set_xlim([0.45, 0.95])
ax[0].set_ylim([0.45, 0.95])

ax[0].set_ylabel('FLARE AFR-like\nancestry estimate', fontsize=9)
ax[1].set_ylabel('FLARE EUR-like\nancestry estimate', fontsize=9)
ax[2].set_ylabel('FLARE EAS-like\nancestry estimate', fontsize=9)

ax[2].set_xlabel('Rye EAS-like ancestry estimate', fontsize=9)
ax[1].set_xlabel('Rye EUR-like ancestry estimate', fontsize=9)
ax[0].set_xlabel('Rye AFR-like ancestry estimate', fontsize=9)
ax[0].annotate("A", (1, 1), (-.3, 0.95), xycoords='axes fraction', fontsize=14, fontweight='bold')
ax[1].annotate("B", (1, 1), (-.3, 0.95), xycoords='axes fraction', fontsize=14, fontweight='bold')
ax[2].annotate("C", (1, 1), (-.33, 0.95), xycoords='axes fraction', fontsize=14, fontweight='bold')

fig.savefig('visualizations/corr_rye_flare_global_ancestry.pdf', bbox_inches='tight', dpi=600)
fig.savefig('visualizations/corr_rye_flare_global_ancestry.png', bbox_inches='tight', dpi=600)

In [ ]:
local_ancestry_fname_pattern = 'results50kb_wo_PEGS/flare/african_american_and_ref_individuals_chr{chrom}.anc_per_pos.phase{phase}.Vindija33.19.bed'
local_ancestry = read_flare(local_ancestry_fname_pattern)
local_ancestry.IID = local_ancestry.IID.values.astype(str)

### Plot introgressed amounts per individual per population

In [ ]:
ibdmix = read_ibdmix_results('results50kb_wo_PEGS/ibdmix_Vindija33.19/ibdmix_results_masked_denisovan_combined_50kb_4.0LOD.bed')
ibdmix_masked = read_ibdmix_results('results50kb_wo_PEGS/ibdmix_Vindija33.19/ibdmix_results_masked_denisovan_combined_50kb_4.0LOD_afr_masked.bed')
ibdmix.replace({"AOUNA": "AOU-NA", 'AOUAFR': 'AOU-AFR', 'PEGS': 'PEGS-EUR', 'AOUEUR': 'AOU-EUR'}, inplace=True)
ibdmix.replace({population: f'1KGP-{population}' for population in ibdmix['pop'].unique() 
                if not population.startswith('AOU-') and population != 'AA'}, inplace=True)

ibdmix_masked.replace({"AOUNA": "AOU-NA", 'AOUAFR': 'AOU-AFR', 'PEGS': 'PEGS-EUR', 'AOUEUR': 'AOU-EUR'},
                      inplace=True)
ibdmix_masked.replace({population: f'1KGP-{population}' for population in ibdmix_masked['pop'].unique()
                       if not population.startswith('AOU-') and population != 'AA'}, inplace=True)
fig, ax = compare_amounts_predicted_introgressed_sequence_by_populations(ibdmix, 'Vindija33.19')
fig1, ax1 = compare_amounts_predicted_introgressed_sequence_by_populations(ibdmix_masked, 
                                                                           'Vindija33.19 AFR-masked')
# fig.savefig('visualizations/amounts_neanderthal_global.pdf', bbox_inches='tight', dpi=600)
# fig.savefig('visualizations/amounts_neanderthal_global.png', bbox_inches='tight', dpi=600)

fig1.savefig('visualizations/amounts_neanderthal_global_afr_masked.pdf', bbox_inches='tight', dpi=600)
fig1.savefig('visualizations/amounts_neanderthal_global_afr_masked.png', bbox_inches='tight', dpi=600)

### Overlap of introgressed segments with local ancestry 

In [ ]:
def get_overlap_local_ancestry_neanderthal_segments(args):
    ibdmix, local_ancestry, iids = args
    afr_enrichment = []
    eur_enrichment = []
    eas_enrichment = []
    columns = local_ancestry.columns.values
    columns = np.concatenate([columns, [col + '_nea' for col in ibdmix.columns.values], ['overlap']])
    for iid in iids:
        c_ibdmix = ibdmix[ibdmix.IID == iid]
        c_local_ancestry = local_ancestry[local_ancestry.IID == iid]
        overlap = BedTool.from_dataframe(c_local_ancestry).intersect(BedTool.from_dataframe(c_ibdmix), 
                                                             wao=True).to_dataframe(names=columns)
        ancestry_prop = (c_local_ancestry.groupby('la')['length'].sum() / 
                         c_local_ancestry.groupby('la')['length'].sum().sum())
        nea_overlap = overlap.groupby('la')['overlap'].sum() / overlap.groupby('la')['overlap'].sum().sum()
        afr_enrichment.append(nea_overlap["AFR"] / ancestry_prop['AFR'])
        eur_enrichment.append(nea_overlap["EUR"] / ancestry_prop['EUR'])
        try:
            eas_enrichment.append(nea_overlap["EAS"] / ancestry_prop['EAS'])
        except KeyError:
            eas_enrichment.append(0.0)
    return (afr_enrichment, eur_enrichment, eas_enrichment)

amr_iids = ibdmix.loc[ibdmix.super_pop == 'AMR', 'IID'].unique()
splits = np.array_split(amr_iids, 16)
ready_to_map = [(ibdmix, local_ancestry, split) for split in splits]
pool = mp.Pool(processes=16)
results = pool.map(get_overlap_local_ancestry_neanderthal_segments, ready_to_map)
pool.close()
pool.join()
afr_enrichment = np.concatenate([res[0] for res in results])
eur_enrichment = np.concatenate([res[1] for res in results])
eas_enrichment = np.concatenate([res[2] for res in results])

In [ ]:
fig, ax = plt.subplots()
ax.hist(afr_enrichment, bins=100, color='red', alpha=0.7, label='AFR', density=True, 
        range=(min([min(afr_enrichment), min(eur_enrichment)]), 
               max([max(afr_enrichment), max(eur_enrichment)])))
ax.hist(eur_enrichment, bins=100, color='blue', alpha=0.7, label='EUR', density=True,
        range=(min([min(afr_enrichment), min(eur_enrichment)]), 
               max([max(afr_enrichment), max(eur_enrichment)])))
# ax.hist(eas_enrichment, bins=100, color='green', alpha=0.7, label='EAS', density=True)
ax.legend()
ax.set_ylabel('Density')
ax.axvline(1, ls='--', color='black')
ax.set_xlabel('Neanderthal ancestry enrichment in local ancestry')
fig.savefig('visualizations/nea_enrichment_in_local_ancestry.pdf')

### Plot introgressed amount vs ancestry proportions

In [ ]:
neanderthal = 'Vindija33.19'
# calculate introgressed amount per individual
amounts = ibdmix.groupby('IID').sum().loc[:, ['length']] / 1e6
# join with admixture proportions
amounts = amounts.join(admixture_proportions)
# select admixed individuals only
amounts = amounts[amounts.population == 'AA']
# regress introgressed amounts against admixture proportions

(slope_afr, intercept_afr, rval_afr, pval_afr, 
 x_model_afr, y_model_afr, ci_afr, pi_afr) = linear_regression(amounts.AFR, amounts['length'])
(slope_eas, intercept_eas, rval_eas, pval_eas, 
 x_model_eas, y_model_eas, ci_eas, pi_eas) = linear_regression(amounts.EAS, amounts['length'])
(slope_eur, intercept_eur, rval_eur, pval_eur, 
 x_model_eur, y_model_eur, ci_eur, pi_eur) = linear_regression(amounts.EUR, amounts['length'])


fig = plt.figure(figsize=(10, 8))
gs = fig.add_gridspec(2, 3)
ax = fig.add_subplot(gs[0, :])
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[1, 1])
ax3 = fig.add_subplot(gs[1, 2])
plt.subplots_adjust(wspace=0.2, hspace=0.4)

fig, ax = compare_amounts_predicted_introgressed_sequence_by_populations(ibdmix, 'Vindija33.19', 
                                                                         ax=ax, fig=fig)



ax3.scatter(amounts.EAS, amounts['length'].values, color='green', s=2, alpha=0.5)
ax3.plot(x_model_eas, y_model_eas, ls='--', color='black')
# ax3.fill_between(x_model_eas, y_model_eas + ci_eas, y_model_eas - ci_eas, color="black", alpha=0.4)
if pval_eas <= 1e-6:
    ax3.annotate('P' + r'$\leq$' + '{:.0e}; '.format(1e-6) + r'$r$' + 
                   '={:.2f}'.format(rval_eas ** 2), (1, 1), (0.5, 0.95), 
                   xycoords='axes fraction', fontsize=8)
else:
    ax3.annotate('P={:.2e}; '.format(pval_eas) + r'$r$' + 
                   '={:.2f}'.format(rval_eas ** 2), (1, 1), (0.5, 0.95), 
                   xycoords='axes fraction', fontsize=8)
# ax[0].plot(x_model_eas, y_model_eas + pi_eas, ls=':', color='black')
# ax[0].plot(x_model_eas, y_model_eas - pi_eas, ls=':', color='black')

ax2.scatter(amounts.EUR, amounts['length'].values, color='blue', s=2, alpha=0.5)
ax2.plot(x_model_eur, y_model_eur, ls='--', color='black')
# ax2.fill_between(x_model_eur, y_model_eur + ci_eur, y_model_eur - ci_eur, color="black", alpha=0.4)
if pval_eur <= 1e-6:
    ax2.annotate('P' + r'$\leq$' + '{:.0e}; '.format(1e-6) + r'$r$' + 
                   '={:.2f}'.format(rval_eur ** 2), (1, 1), (0.05, 0.95), 
                   xycoords='axes fraction', fontsize=8)
else:
    ax2.annotate('P={:.2e}; '.format(pval_eur) + r'$r$' + 
                   '={:.2f}'.format(rval_eur ** 2), (1, 1), (0.05, 0.95), 
                   xycoords='axes fraction', fontsize=8)
# ax[1].plot(x_model_eur, y_model_eur + pi_eur, ls=':', color='black')
# ax[1].plot(x_model_eur, y_model_eur - pi_eur, ls=':', color='black')

ax1.scatter(amounts.AFR, amounts['length'].values, color='red', s=2, alpha=0.5)
ax1.plot(x_model_afr, y_model_afr, ls='--', color='black')
# ax1.fill_between(x_model_afr, y_model_afr + ci_afr, y_model_afr - ci_afr, color="black", alpha=0.4)
if pval_afr <= 1e-6:
    ax1.annotate('P' + r'$\leq$' + '{:.0e}; '.format(1e-6) + r'$r$' + 
                   '={:.2f}'.format(rval_afr ** 2), (1, 1), (0.5, 0.95), 
                   xycoords='axes fraction', fontsize=8)
else:
    ax1.annotate('P={:.2e}; '.format(pval_afr) + r'$r$' + 
                   '={:.2f}'.format(rval_afr ** 2), (1, 1), (0.5, 0.95), 
                   xycoords='axes fraction', fontsize=8)
# ax[2].plot(x_model_afr, y_model_afr + pi_afr, ls=':', color='black')
# ax[2].plot(x_model_afr, y_model_afr - pi_afr, ls=':', color='black')

ax1.set_ylabel(f'{neanderthal} introgressed\nper individual (Mb)')
ax3.set_xlabel('EAS-/NA-like ancestry proportion')
ax2.set_xlabel('EUR-like ancestry proportion')
ax1.set_xlabel('AFR-like ancestry proportion')
yticklabels = ax1.get_yticklabels()
ax2.set_yticklabels([])
ax3.set_yticklabels([])
ax.annotate("A", (1, 1), (-0.08, 0.92), xycoords='axes fraction', fontsize=16, fontweight='bold')
ax1.annotate("B", (1, 1), (-0.28, 0.92), xycoords='axes fraction', fontsize=16, fontweight='bold')
ax2.annotate("C", (1, 1), (-0.15, 0.92), xycoords='axes fraction', fontsize=16, fontweight='bold')
ax3.annotate("D", (1, 1), (-0.15, 0.92), xycoords='axes fraction', fontsize=16, fontweight='bold')

fig.savefig("visualizations/figure2.pdf", bbox_inches='tight', dpi=600)
fig.savefig("visualizations/figure2.png", bbox_inches='tight', dpi=600)

# ax1.set_yticklabels(yticklabelslabels)

In [ ]:
def get_average_amount_per_super_pop(df):
    super_pop_indv_mapping = df.drop_duplicates("IID").loc[:, ["IID", 'super_pop']].set_index('IID')
    # groupby individual
    amount_introgression_per_indv = df.loc[:, ['IID', 'length']].groupby('IID').sum() / 1e6
    amount_introgression_superpop_per_indv = super_pop_indv_mapping.join(amount_introgression_per_indv)
    mean_introgression_per_superpop = amount_introgression_superpop_per_indv.groupby('super_pop').mean()
    return mean_introgression_per_superpop
get_average_amount_per_super_pop(ibdmix)

In [ ]:
get_average_amount_per_super_pop(ibdmix_masked)

### Compare unique introgressed sequence

In [ ]:
def get_total_unique_sequence_super_pop(df, super_pop, bootstrap=False, n_indvs=13844):
    df = df[df.super_pop == super_pop]
    if bootstrap:
        iids = df[df.super_pop == super_pop].IID.unique()
        b_iids = np.random.choice(iids, size=(n_indvs, 100))
        df.set_index('IID', inplace=True)
        unique = []
        for i in tqdm(range(100)):
            c_df = df.loc[b_iids[:, i]].sort_values(['chrom', 'start'])
            c_unique = BedTool.from_dataframe(c_df).merge().to_dataframe()
            c_unique['length'] = c_unique.end - c_unique.start
            unique.append(c_unique['length'].sum() / 1e6)
    else:
        unique = BedTool.from_dataframe(df).merge().to_dataframe()
        unique['length'] = unique.end - unique.start
        unique = unique['length'].sum() / 1e6
    return unique


print("EUR: " + str(get_total_unique_sequence_super_pop(ibdmix_masked[ibdmix_masked.super_pop =='EUR'].groupby('IID').sample(frac=0.233).sort_values(['chrom', 'start']), 'EUR')))
# print("Admixed: " + str(get_total_unique_sequence_super_pop(ibdmix_masked, 'AMR')))

In [ ]:
unique_admixed = get_total_unique_sequence_super_pop(ibdmix_masked, 'AMR', bootstrap=True)
print("Admixed: " + str(np.mean(unique_admixed)) + "(" + str(np.percentile(unique_admixed, 2.5)) + "-" +
      str(np.percentile(unique_admixed, 97.5)) + ")")


In [ ]:
ibdmix_masked[ibdmix_masked.super_pop == 'AMR'].IID.value_counts().mean() /ibdmix_masked[ibdmix_masked.super_pop == 'EUR'].IID.value_counts().mean()

In [ ]:
def get_average_amount_per_pop(df):
    pop_indv_mapping = df.drop_duplicates("IID").loc[:, ["IID", 'pop']].set_index('IID')
    # groupby individual
    amount_introgression_per_indv = df.loc[:, ['IID', 'length']].groupby('IID').sum() / 1e6
    amount_introgression_pop_per_indv = pop_indv_mapping.join(amount_introgression_per_indv)
    mean_introgression_per_pop = amount_introgression_pop_per_indv.groupby('pop').mean()
    return mean_introgression_per_pop


unique_eur = BedTool.from_dataframe(ibdmix_masked[ibdmix_masked.super_pop == 'EUR']).intersect(
    BedTool.from_dataframe(ibdmix_masked[ibdmix_masked.super_pop == 'AMR']), 
                           v=True).to_dataframe(names=ibdmix_masked.columns)
get_average_amount_per_pop(unique_eur)

In [ ]:
def get_total_unique_sequence_pop(df, population, bootstrap=False, n_indvs=None):
    df = df[df['pop'] == population]
    if bootstrap:
        iids = df.IID.unique()
        b_iids = np.random.choice(iids, size=(n_indvs, 100))
        df.set_index('IID', inplace=True)
        unique = []
        for i in tqdm(range(100)):
            c_df = df.loc[b_iids[:, i]].sort_values(['chrom', 'start'])
            c_unique = BedTool.from_dataframe(c_df).merge().to_dataframe()
            c_unique['length'] = c_unique.end - c_unique.start
            unique.append(c_unique['length'].sum() / 1e6)
    else:
        unique = BedTool.from_dataframe(df).merge().to_dataframe()
        unique['length'] = unique.end - unique.start
        unique = unique['length'].sum() / 1e6
    return unique

print("AOU-EUR: " + str(get_total_unique_sequence_pop(unique_eur, 'AOU-EUR')))
print("PEGS-EUR: " + str(get_total_unique_sequence_pop(unique_eur, 'PEGS-EUR')))
print("CEU: " + str(get_total_unique_sequence_pop(unique_eur, 'CEU')))
print("GBR: " + str(get_total_unique_sequence_pop(unique_eur, 'GBR')))
print("IBS: " + str(get_total_unique_sequence_pop(unique_eur, 'IBS')))
print("TSI: " + str(get_total_unique_sequence_pop(unique_eur, 'TSI')))
print("FIN: " + str(get_total_unique_sequence_pop(unique_eur, 'FIN')))


In [ ]:
unique_aou = get_total_unique_sequence_pop(unique_eur, 'AOU-EUR', bootstrap=True, n_indvs=91)
unique_pegs = get_total_unique_sequence_pop(unique_eur, 'PEGS-EUR', bootstrap=True, n_indvs=91)
unique_ceu = get_total_unique_sequence_pop(unique_eur, 'CEU', bootstrap=True, n_indvs=91)
unique_gbr = get_total_unique_sequence_pop(unique_eur, 'GBR', bootstrap=True, n_indvs=91)
unique_ibs = get_total_unique_sequence_pop(unique_eur, 'IBS', bootstrap=True, n_indvs=91)
unique_tsi = get_total_unique_sequence_pop(unique_eur, 'TSI', bootstrap=True, n_indvs=91)
unique_fin = get_total_unique_sequence_pop(unique_eur, 'FIN', bootstrap=True, n_indvs=91)

for population, unique in zip(['AOU-EUR', 'PEGS-EUR', 'CEU', "GBR", "IBS", "TSI", "FIN"],
                              [unique_aou, unique_pegs, unique_ceu, unique_gbr, unique_ibs, 
                               unique_tsi, unique_fin]):
    print(f"{population}: {np.mean(unique)} ({np.percentile(unique, 2.5)} - "+
          f"{np.percentile(unique, 97.5)})")


### Compare expected versus observed amounts

In [ ]:
def calculate_expected_introgression(admixture_proportions, ibdmix):
    # calculate introgressed amount per individual
    amounts = ibdmix.groupby('IID').sum().loc[:, ['length']] / 1e6
    # join with admixture proportions
    amounts = amounts.join(admixture_proportions)
    # select admixed individuals only
    amounts = amounts[amounts.population == 'AA']
    super_pop_indv_mapping = ibdmix.drop_duplicates("IID").loc[:, ["IID", 'super_pop']].set_index('IID')
    # groupby individual
    amount_introgression_per_indv = ibdmix.loc[:, ['IID', 'length']].groupby('IID').sum() / 1e6
    # calculate mean per continental population
    amount_introgression_superpop_per_indv = super_pop_indv_mapping.join(amount_introgression_per_indv)
    mean_introgression_per_superpop = amount_introgression_superpop_per_indv.groupby('super_pop').mean()
    if not 'AFR' in mean_introgression_per_superpop.index:
        mean_introgression_per_superpop.loc['AFR', 'length'] = 0.0
    if not 'EUR' in mean_introgression_per_superpop.index:
        mean_introgression_per_superpop.loc['EUR', 'length'] = 0.0
    if not 'EAS' in mean_introgression_per_superpop.index:
        mean_introgression_per_superpop.loc['EAS', 'length'] = 0.0
    amounts['expected_introgression'] = (amounts.AFR * 
                                         mean_introgression_per_superpop.loc['AFR', 'length'] + 
                                         amounts.EUR * 
                                         mean_introgression_per_superpop.loc['EUR', 'length'] +
                                         amounts.EAS * 
                                         mean_introgression_per_superpop.loc['EAS', 'length'])
        
    return amounts

def plot_observed_vs_expected_amounts(amounts, color_scatter='lightblue', color_reg='blue', ax=None):
    # do regression
    (slope, intercept, rval, pval, x_model, y_model, 
     ci, pi) = linear_regression(amounts.expected_introgression, amounts['length'], full_range=True)
    if ax is None:
        fig, ax = plt.subplots()
    amounts =  amounts.copy()
    amounts.reset_index(inplace=True)
    xy = np.vstack([amounts.expected_introgression, amounts['length']])
    z = gaussian_kde(xy)(xy)
    # Sort the points by density, so that the densest points are plotted last
    idx = z.argsort()
    x, y, z = amounts.loc[idx, 'expected_introgression'], amounts.loc[idx, 'length'], z[idx]
    cax = ax.scatter(x, y, c=z, s=2, cmap=color_scatter, 
                     norm=LogNorm(vmin=z.min(), vmax=z.max()))
#     ax.scatter(amounts.expected_introgression.values, amounts['length'].values, c=color_scatter, s=2)
    if intercept > 0:
        ax.plot(x_model, y_model, ls='--', color=color_reg,
                label=f'y={np.round(slope, 2)}x + {np.round(intercept, 2)}')
    else:
        ax.plot(x_model, y_model, ls='--', color=color_reg,
                label=f'y={np.round(slope, 2)}x - {np.round(np.abs(intercept), 2)}')
#     ax.fill_between(x_model, y_model + ci, y_model - ci, color=color_reg, alpha=0.4)
    if pval <= 1e-6:
        ax.annotate('P' + r'$\leq$' + '{:.0e}; '.format(1e-6) + r'$r$' + 
                    '={:.2f}'.format(rval ** 2), (1, 1), (0.05, 0.93), 
                   xycoords='axes fraction')
    else:
        ax.annotate('P={:.2e}; '.format(pval) + r'$r$' + 
                    '={:.2f}'.format(rval ** 2), (1, 1), (0.05, 0.93), 
                       xycoords='axes fraction')
    ax.plot(x_model, x_model, ls='--', c='black', label='y=x')
    ax.legend(bbox_to_anchor=(0.5, -.27), loc='center', ncol=2)
    # ax.plot(x_model, y_model + pi, ls=':', color='blue')
    # ax.plot(x_model, y_model - pi, ls=':', color='blue')
    ax.set_xlabel('Expected introgression per individual (Mb)')
    ax.set_ylabel('Observed introgression\nper individual (Mb)')
    min_lim = max([min([amounts.expected_introgression.min(), amounts['length'].min()]) - 1, 0])
    max_lim = max([amounts.expected_introgression.max(), amounts['length'].max()]) + 1
    ax.set_xlim([min_lim, max_lim])
    ax.set_ylim([min_lim, max_lim])
    ax.set_aspect('equal')
    
    return ax, cax

def plot_hist_of_ratios_of_obs_vs_exp_amounts(df, color, simulation_ratios, ax=None,
                                              bins=np.linspace(-.2, .5, 200), 
                                              color_sim='gray'):
    if ax is None:
        fig, ax = plt.subplots()
    
    for i in range(simulation_ratios.shape[0]):
        if simulation_ratios[i].shape[0] < 900:
#             print(i)
            continue
        ax.hist(np.log10(simulation_ratios[i].astype(float)), bins=bins, histtype='step', color=color_sim,
                weights=np.repeat(1 / simulation_ratios[i].shape[0], simulation_ratios[i].shape[0]), 
                alpha=0.1)
    ax.hist(np.log10(df['length'] / df['expected_introgression']), bins=bins, histtype='step', color=color,
            weights=np.repeat(1 / df.shape[0], df.shape[0]), alpha=1)
    ax.axvline(0, ls='--', color='black')
#     test = ttest_1samp(np.log(df['length'] / df['expected_introgression']), 0)
#     if test.pvalue <= 1e-6:
#         ax.annotate('P' r'$\leq$' + '{:.0e}'.format(1e-6), (1, 1), (0.8, 0.95), 
#                     xycoords='axes fraction')
#     else:
#         ax.annotate('P={:.3e}'.format(test.pvalue), (1, 1), (0.8, 0.95), 
#                    xycoords='axes fraction')
    ax.axvline(0, ls='--', color='black')
    ax.set_xlabel(r'$log10(\frac{Observed\ introgression\ in\ Mb}{Expected\ introgression\ in\ Mb})$')
    return ax

def plot_hist_of_diff_of_obs_vs_exp_amounts(df, color, simulation_diffs, ax=None,
                                            bins=np.linspace(-.2, .5, 200), 
                                            color_sim='gray'):
    if ax is None:
        fig, ax = plt.subplots()
    pvals = []
    for i in range(simulation_diffs.shape[0]):
        # divide diff by simulated genome size
        ax.hist(simulation_diffs[i] / (903043000 / 1e6) * 100, bins=bins, histtype='step', color=color_sim,
                weights=np.repeat(1 / simulation_diffs[i].shape[0], simulation_diffs[i].shape[0]), 
                alpha=0.5)
        pvals.append(mannwhitneyu(simulation_diffs[i].astype(float) / (903043000 / 1e6) * 100, 
                                   (df['length'].values - 
                                    df['expected_introgression'].values) / (2875001522 / 1e6) * 100).pvalue)
    # divide diff by autosomal genome size
    ax.hist((df['length'] - df['expected_introgression']) / (2875001522 / 1e6) * 100, bins=bins, 
            histtype='step', color=color,
            weights=np.repeat(1 / df.shape[0], df.shape[0]), alpha=1)
    ax.axvline(0, ls='--', color='black')
    ax.set_xlabel('Observed - expected\nNeanderthal admixture fraction (%)')
#     test = ttest_1samp(np.log(df['length'] / df['expected_introgression']), 0)
#     if hmean(pvals) <= 1e-6:
#         ax.annotate('P' r'$\leq$' + '{:.0e}'.format(1e-6), (1, 1), (0.6, 0.93), 
#                     xycoords='axes fraction')
#     else:
#         ax.annotate('P={:.3e}'.format(hmean(pvals)), (1, 1), (0.6, 0.93), 
#                    xycoords='axes fraction')
    ax.axvline(0, ls='--', color='black')
    return ax

def load_simulations(prefix):
    ratios = load_npz(f'{prefix}_ratios.npz')
    ratios_short = load_npz(f'{prefix}_ratios_short_segments.npz')
    ratios_medium = load_npz(f'{prefix}_ratios_medium_segments.npz')
    ratios_long = load_npz(f'{prefix}_ratios_long_segments.npz')

    differences = load_npz(f'{prefix}_differences.npz')
    differences_short = load_npz(f'{prefix}_differences_short_segments.npz')
    differences_medium = load_npz(f'{prefix}_differences_medium_segments.npz')
    differences_long = load_npz(f'{prefix}_differences_long_segments.npz')

    slopes = load_npz(f'{prefix}_slopes.npz')
    intercepts = load_npz(f'{prefix}_intercepts.npz')
    try:
        lengths = load_npz(f'{prefix}_segment_lengths.npz')
        lengths = np.concatenate(lengths) / 1000
    except:
        print(f"couldn't load {prefix}_segment_lengths.npz")
        lengths = []
    try:
        lods = load_npz(f'{prefix}_segment_lods.npz')
        lods = np.concatenate(lods)
    except:
        print(f"couldn't load {prefix}_segment_lods.npz")
        lods = []
    return (ratios, ratios_short, ratios_medium, ratios_long, differences, differences_short, 
            differences_medium, differences_long, slopes, intercepts, lengths, lods) 

# def plot_scatter_and_simulated_hist(amounts, simulations_simple, simulations_full,
def plot_scatter_and_simulated_hist(amounts, simulations,
                                    histrange=(-.5, .5), color_scatter='royalblue', color_reg='blue',
                                    color_sim='blue', plot_difference=False):
    fig = plt.figure(figsize=(10, 6))
    gs_outer = fig.add_gridspec(1, 2, width_ratios=[5, 6], wspace=0.35)
    gs2 = gridspec.GridSpecFromSubplotSpec(44, 20, subplot_spec = gs_outer[1], wspace=0.15)

    ax = fig.add_subplot(gs_outer[0])
    
    ax1 = fig.add_subplot(gs2[8:36, 3:])
    bins = np.linspace(histrange[0], histrange[1], 200)
    ax, cax = plot_observed_vs_expected_amounts(amounts, ax=ax, color_scatter=color_scatter, 
                                           color_reg=color_reg)
    divider = make_axes_locatable(ax)
    c_ax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(cax, cax=c_ax, ax=ax)
    cbar.set_label('Density')
    if plot_difference:
        ax1 = plot_hist_of_diff_of_obs_vs_exp_amounts(amounts, color_sim, simulations,
                                                        ax=ax1, bins=bins)
    else:
        ax1 = plot_hist_of_ratios_of_obs_vs_exp_amounts(amounts, color_sim, simulations,
                                                        ax=ax1, bins=bins)

    handles = [Line2D([0], [0], ls='-', color='gray', label='Simulations'),
               Line2D([0], [0], ls='-', color=color_sim, label='AOU-Admixed')]
    ax1.legend(handles=handles, bbox_to_anchor=(0.5, -.2), ncol=2, loc='upper center')
    ax1.set_ylabel('Density')




    ax.annotate("A", (1, 1), (-.22, 0.95), xycoords='axes fraction', fontsize=14, fontweight='bold')
    ax1.annotate("B", (1, 1), (-.2, 0.95), xycoords='axes fraction', fontsize=14, fontweight='bold')
    return fig, ax, ax1


amounts = calculate_expected_introgression(admixture_proportions, ibdmix)

(simulations_ratios, simulations_ratios_short, 
 simulations_ratios_medium, simulations_ratios_long, 
 simulations_differences, simulations_differences_short, 
 simulations_differences_medium, 
 simulations_differences_long, simulations_slopes, 
 simulations_intercepts, simulations_lengths, 
 simulations_lods) = load_simulations('simulations')



fig, ax, ax1 = plot_scatter_and_simulated_hist(amounts, 
                                               simulations_differences, 
                                               color_scatter='Purples', color_reg='mediumvioletred', 
                                               color_sim='purple', plot_difference=True, 
                                               histrange=(-.5, 1))

fig.savefig('visualizations/nea_enrichment.pdf', bbox_inches='tight', dpi=600)
fig.savefig('visualizations/nea_enrichment.png', bbox_inches='tight', dpi=600)


#### Mask African component to check if ILS might be driving it

In [ ]:
amounts_masked = calculate_expected_introgression(admixture_proportions, ibdmix_masked)

(simulations_afr_masked_ratios, simulations_afr_masked_ratios_short, 
 simulations_afr_masked_ratios_medium, simulations_afr_masked_ratios_long, 
 simulations_afr_masked_differences, simulations_afr_masked_differences_short, 
 simulations_afr_masked_differences_medium, 
 simulations_afr_masked_differences_long, simulations_afr_masked_slopes, 
 simulations_afr_masked_intercepts, simulations_afr_masked_lengths, 
 simulations_afr_masked_lods) = load_simulations('simulations_afr_masked')

fig, ax, ax1 = plot_scatter_and_simulated_hist(amounts_masked, 
                                               simulations_afr_masked_differences, 
                                               color_scatter='Reds', color_reg='darksalmon', 
                                               color_sim='red', plot_difference=True, 
                                               histrange=(-.2, 0.5))

fig.savefig('visualizations/figure3.pdf', bbox_inches='tight', dpi=600)
fig.savefig('visualizations/figure3.png', bbox_inches='tight', dpi=600)


In [ ]:
np.mean((amounts_masked['length'] - amounts_masked['expected_introgression']))

### Load recombination filter

In [ ]:
genetic_maps = []
for chrom in np.arange(1, 23):
    genetic_map = pd.read_csv(f'data/hapmap_genetic_map/genetic_map_Hg38_chr{chrom}.txt', sep='\t')
    genetic_map.rename({'Chromosome': 'chrom_map', 'Position(bp)': 'end_map', 'Rate(cM/Mb)': 'rate'}, axis=1, 
                       inplace=True)
    genetic_map['start_map'] = np.concatenate([[0], genetic_map.end_map[:-1]])
    genetic_map = genetic_map.loc[:, ['chrom_map', 'start_map', 'end_map', 'rate']]
    genetic_maps.append(genetic_map)
genetic_map = pd.concat(genetic_maps)
windows = BedTool('data/reference/hg38.chrom.sizes.bed').window_maker(w=300000, 
                                                                      g='data/reference/genomefile_hg38.bed')
columns = ['chrom', 'start', 'end']
columns.extend(genetic_map.columns.values.tolist())
columns.append('overlap')
rec_windows = windows.intersect(BedTool.from_dataframe(genetic_map), wao=True).to_dataframe(names=columns)
rec_windows.rate = np.where(rec_windows.rate == '.', '0', rec_windows.rate)
rec_windows.rate = rec_windows.rate.astype(float)
rec_windows['rate'] *= rec_windows['overlap']
rec_windows = rec_windows.loc[:,['chrom', 'start', 'end', 
                                 'rate', 'overlap']].groupby(['chrom', 'start', 'end']).sum()
rec_windows.rate /= rec_windows.overlap
rec_windows.reset_index(inplace=True)
rec_windows.fillna(0, inplace=True)
low_rec_thresh = np.percentile(rec_windows.rate.values, 33)
high_rec_thresh = np.percentile(rec_windows.rate.values, 66)
pass_rec_filter = rec_windows.loc[(rec_windows.rate >= low_rec_thresh) & 
                                  (rec_windows.rate <= high_rec_thresh), ['chrom', 'start', 'end']]
pass_rec_filter = BedTool.from_dataframe(pass_rec_filter).merge().to_dataframe()

In [ ]:
ibdmix_recomb = BedTool.from_dataframe(ibdmix).intersect(BedTool.from_dataframe(pass_rec_filter), 
                                                             f=1).to_dataframe(names=ibdmix.columns)
amounts_recomb = calculate_expected_introgression(admixture_proportions, ibdmix_recomb)


(simulations_recomb_masked_ratios, simulations_recomb_masked_ratios_short, 
 simulations_recomb_masked_ratios_medium, simulations_recomb_masked_ratios_long, 
 simulations_recomb_masked_differences, simulations_recomb_masked_differences_short, 
 simulations_recomb_masked_differences_medium, 
 simulations_recomb_masked_differences_long, simulations_recomb_masked_slopes, 
 simulations_recomb_masked_intercepts, simulations_recomb_masked_lengths, 
 simulations_recomb_masked_lods) = load_simulations('simulations_recomb_masked')

fig, ax, ax1 = plot_scatter_and_simulated_hist(amounts_recomb, 
                                               simulations_recomb_masked_differences, 
                                               color_scatter='Oranges', color_reg='orange', 
                                               color_sim='darkorange', plot_difference=True, 
                                               histrange=(-.2, .5))
fig.savefig('visualizations/nea_enrichment_masked_recomb.pdf', bbox_inches='tight', dpi=600)
fig.savefig('visualizations/nea_enrichment_masked_recomb.png', bbox_inches='tight', dpi=600)

#### Apply both filters

In [ ]:
ibdmix_afr_recomb = BedTool.from_dataframe(ibdmix_masked).intersect(BedTool.from_dataframe(pass_rec_filter), 
                                                         f=1).to_dataframe(names=ibdmix_masked.columns)
amounts_afr_recomb = calculate_expected_introgression(admixture_proportions, ibdmix_afr_recomb)

(simulations_afr_recomb_masked_ratios, simulations_afr_recomb_masked_ratios_short, 
 simulations_afr_recomb_masked_ratios_medium, simulations_afr_recomb_masked_ratios_long, 
 simulations_afr_recomb_masked_differences, simulations_afr_recomb_masked_differences_short, 
 simulations_afr_recomb_masked_differences_medium, 
 simulations_afr_recomb_masked_differences_long, simulations_afr_recomb_masked_slopes, 
 simulations_afr_recomb_masked_intercepts, simulations_afr_recomb_masked_lengths, 
 simulations_afr_recomb_masked_lods) = load_simulations('simulations_afr_recomb_masked')

fig, ax, ax1 = plot_scatter_and_simulated_hist(amounts_afr_recomb, 
                                               simulations_afr_recomb_masked_differences, 
                                               color_scatter='Greens', color_reg='forestgreen', 
                                               color_sim='forestgreen', plot_difference=True, 
                                               histrange=(-.2, .5))

### Explore the segments that are removed by the African mask

In [ ]:
ibdmix_removed = BedTool.from_dataframe(ibdmix).intersect(
    BedTool.from_dataframe(ibdmix[ibdmix.super_pop == 'AFR']), 
    u=True).to_dataframe(names=ibdmix.columns)

In [ ]:
columns = rec_windows.columns.tolist()
columns.append('n_overlap')
rec_afr_mask = BedTool.from_dataframe(rec_windows).intersect(
    BedTool.from_dataframe(ibdmix_removed[ibdmix_removed['pop'] == 'AA']), C=True).to_dataframe(names=columns)
rec_non_afr_mask = BedTool.from_dataframe(rec_windows).intersect(
    BedTool.from_dataframe(ibdmix_masked[ibdmix_masked['pop'] == 'AA']), C=True).to_dataframe(names=columns)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3), sharey=True)

ax[0].step(np.concatenate([[ibdmix_removed['length'].min() / 1000], np.sort(ibdmix_removed['length']) / 1000]), 
        np.arange(0, ibdmix_removed['length'].shape[0] +1) / float(ibdmix_removed['length'].shape[0]), 
        color='purple', label='Segments removed with AFR mask')
ax[0].step(np.concatenate([[ibdmix_masked['length'].min() / 1000], np.sort(ibdmix_masked['length']) / 1000]), 
        np.arange(0, ibdmix_masked['length'].shape[0] +1) / float(ibdmix_masked['length'].shape[0]), 
        color='red', label='Segments retained')
ax[0].set_xlabel('Segment length in kb')
ax[0].set_ylabel("CDF")
ax[0].set_xlim([50, 250])
_, pval = mannwhitneyu(ibdmix_removed['length'], ibdmix_masked['length'])
if pval <= 1e-6:
    ax[0].annotate('P' + r'$\leq$' + '{:.0e}'.format(1e-6), (1, 1), (0.5, 0.05), 
               xycoords='axes fraction')
else:
    ax[0].annotate('P = {:.2e}'.format(pval), (1, 1), (0.5, 0.05), 
               xycoords='axes fraction')



ax[1].step(np.concatenate([[ibdmix_removed['LOD'].min()], np.sort(ibdmix_removed['LOD'])]), 
        np.arange(0, ibdmix_removed['LOD'].shape[0] +1) / float(ibdmix_removed['LOD'].shape[0]), 
        color='purple', label='Segments removed with AFR mask')
ax[1].step(np.concatenate([[ibdmix_masked['LOD'].min()], np.sort(ibdmix_masked['LOD'])]), 
        np.arange(0, ibdmix_masked['LOD'].shape[0] +1) / float(ibdmix_masked['LOD'].shape[0]), 
        color='red', label='Segments retained')
ax[1].legend(bbox_to_anchor=(0.5, -.2), loc='upper center', ncol=2)
ax[1].set_xlabel('LOD score')
ax[1].set_xlim(4, 200)
_, pval = mannwhitneyu(ibdmix_removed['LOD'], ibdmix_masked['LOD'])
if pval <= 1e-6:
    ax[1].annotate('P' + r'$\leq$' + '{:.0e}'.format(1e-6), (1, 1), (0.5, 0.05), 
               xycoords='axes fraction')
else:
    ax[1].annotate('P = {:.2e}'.format(pval), (1, 1), (0.5, 0.05), 
               xycoords='axes fraction')

ax[2].step(np.concatenate([[rec_afr_mask.rate.values.min()], np.sort(rec_afr_mask.rate.values)]), 
        np.concatenate([[0], (np.cumsum(rec_afr_mask.n_overlap.values[np.argsort(rec_afr_mask.rate)]) / 
                              rec_afr_mask.n_overlap.values.sum())]), 
        color='purple', label='Segments removed with AFR mask')
# ax.hist(rec_afr_mask.rate, bins=100, color='red', cumulative=True, histtype='step', 
#         weights=rec_afr_mask.n_overlap.values / rec_afr_mask.n_overlap.values.sum(),
#         label='Segments removed with AFR mask',range=(0, 5))

ax[2].step(np.concatenate([[rec_non_afr_mask.rate.values.min()], np.sort(rec_non_afr_mask.rate.values)]), 
        np.concatenate([[0], (np.cumsum(rec_non_afr_mask.n_overlap.values[np.argsort(rec_non_afr_mask.rate)]) / 
                              rec_non_afr_mask.n_overlap.values.sum())]), 
        color='red', label='Segments retained')
# ax.hist(rec_non_afr_mask.rate, bins=100, color='blue', cumulative=True, histtype='step', 
#         weights=rec_non_afr_mask.n_overlap.values / rec_non_afr_mask.n_overlap.values.sum(), 
#         label='Segments retained', range=(0, 5))
# ax.legend(bbox_to_anchor=(0.5, -.12), loc='upper center', ncol=2)
ax[2].set_xlabel('Recombination rate (cM/Mb)')
_, pval = mannwhitneyu(rec_afr_mask.rate.values * rec_afr_mask.n_overlap.values / rec_afr_mask.n_overlap.values.sum(), 
                       rec_non_afr_mask.rate.values * rec_non_afr_mask.n_overlap.values / rec_non_afr_mask.n_overlap.values.sum())
if pval <= 1e-6:
    ax[2].annotate('P' + r'$\leq$' + '{:.0e}'.format(1e-6), (1, 1), (0.5, 0.05), 
               xycoords='axes fraction')
else:
    ax[2].annotate('P = {:.2e}'.format(pval), (1, 1), (0.5, 0.05), 
               xycoords='axes fraction')
ax[0].annotate("A", (1, 1), (-0.3, 0.92), xycoords='axes fraction', fontsize=14, fontweight='bold')
ax[1].annotate("B", (1, 1), (-0.15, 0.92), xycoords='axes fraction', fontsize=14, fontweight='bold')
ax[2].annotate("C", (1, 1), (-0.15, 0.92), xycoords='axes fraction', fontsize=14, fontweight='bold')
fig.savefig('visualizations/segments_afr_mask.pdf', dpi=600, bbox_inches='tight')
fig.savefig('visualizations/segments_afr_mask.png', dpi=600, bbox_inches='tight')

In [ ]:
mannwhitneyu(ibdmix_removed['length'], ibdmix_masked['length'], alternative='less')

#### Compare introgression frequencies in different Eurpean reference populations

In [ ]:
from scipy.stats import t

def get_introgression_freq(df, n_indv, cols):
    df_freq = BedTool('data/reference/hg38_windowed_w_50000_s_10000.bed').intersect(
        BedTool.from_dataframe(df), wo=True).to_dataframe(names=cols)
    df_freq = df_freq.loc[:, ['chrom', 'start', 
                              'end', 'overlap']].groupby(['chrom', 'start', 'end']).sum() / 50000 / n_indv
#     df_freq += 2
#     df_freq /= (n_indv + 4)
    df_freq.rename({'overlap': 'freq'}, axis=1, inplace=True)
    return df_freq

# ibdmix_pegs_eur = ibdmix[ibdmix['pop'] == 'PEGS']
# ibdmix_aou_eur = ibdmix[ibdmix['pop'] == 'AOUEUR']
# ibdmix_1kgp_eur = ibdmix[(ibdmix['pop'] != 'PEGS') & 
#                          (ibdmix['pop'] != 'AOUEUR') &
#                          (ibdmix['super_pop'] == 'EUR')]

# ibdmix_masked_pegs_eur = ibdmix_masked[ibdmix_masked['pop'] == 'PEGS-EUR']
ibdmix_masked_aou_eur = ibdmix_masked[ibdmix_masked['pop'] == 'AOU-EUR']
ibdmix_masked_1kgp_eur = ibdmix_masked[(ibdmix_masked['pop'] != 'PEGS-EUR') & 
                                       (ibdmix_masked['pop'] != 'AOU-EUR') &
                                       (ibdmix_masked['super_pop'] == 'EUR')]


n_indv_aou = 10000
n_indv_1kgp = 503
# n_indv_pegs = 3341
columns = ['chrom', 'start', 'end']
columns.extend([col if col != 'chrom' and col != 'start' and col != 'end' else col +'_r' 
                for col in ibdmix_masked.columns])
columns.append('overlap')
# introgression_freq_pegs = get_introgression_freq(ibdmix_masked_pegs_eur, n_indv_pegs, columns)
introgression_freq_aou = get_introgression_freq(ibdmix_masked_aou_eur, n_indv_aou, columns)
introgression_freq_1kgp = get_introgression_freq(ibdmix_masked_1kgp_eur, n_indv_1kgp, columns)
# introgression_freq_pegs = get_introgression_freq(ibdmix_pegs_eur, n_indv_pegs, columns)
# introgression_freq_aou = get_introgression_freq(ibdmix_aou_eur, n_indv_aou, columns)
# introgression_freq_1kgp = get_introgression_freq(ibdmix_1kgp_eur, n_indv_1kgp_eur, columns)


# introgression_freq_pegs_aou = introgression_freq_pegs.join(introgression_freq_aou, 
#                                                            lsuffix='_pegs', rsuffix='_aou',
#                                                            how='outer').fillna(0.0)
# introgression_freq_pegs_1kgp = introgression_freq_pegs.join(introgression_freq_1kgp,
#                                                             lsuffix='_pegs', rsuffix='_1kgp',
#                                                             how='outer').fillna(0.0)
introgression_freq_aou_1kgp = introgression_freq_aou.join(introgression_freq_1kgp, 
                                                          lsuffix='_aou', rsuffix='_1kgp',
                                                          how='outer').fillna(0.0)


# (slope_pegs_aou, intercept_pegs_aou, rval_pegs_aou, pval_pegs_aou, x_model_pegs_aou, y_model_pegs_aou,
#  ci_pegs_aou, pi_pegs_aou) = linear_regression(introgression_freq_pegs_aou.freq_aou.values,
#                                                introgression_freq_pegs_aou.freq_pegs.values,
#                                                full_range=True)

(slope_aou_1kgp, intercept_aou_1kgp, rval_aou_1kgp, pval_aou_1kgp, x_model_aou_1kgp, y_model_aou_1kgp,
 ci_aou_1kgp, pi_aou_1kgp) = linear_regression(introgression_freq_aou_1kgp.freq_aou.values,
                                               introgression_freq_aou_1kgp.freq_1kgp.values,
                                               full_range=True)

# (slope_pegs_1kgp, intercept_pegs_1kgp, rval_pegs_1kgp, pval_pegs_1kgp, x_model_pegs_1kgp, y_model_pegs_1kgp,
#  ci_pegs_1kgp, pi_pegs_1kgp) = linear_regression(introgression_freq_pegs_1kgp.freq_pegs.values,
#                                                  introgression_freq_pegs_1kgp.freq_1kgp.values,
#                                                  full_range=True)



# fig, ax = plt.subplots(figsize=(4, 4))
# fig, ax = plt.subplots(1, 3, figsize=(8,5), sharey=True, sharex=True)
fig, ax = plt.subplots(1, 1, figsize=(4,4))

# plt.subplots_adjust(wspace=0.3)
# ax[0].scatter(introgression_freq_pegs_aou.freq_aou, introgression_freq_pegs_aou.freq_pegs, s=2)
# ax[0].plot([0, 1], [0, 1], ls='--', color='black', transform=ax[0].transAxes)
# ax[0].set_xlabel('AOU-EUR (10,000)')
# ax[0].set_ylabel('PEGS-EUR (3,341)')
# ax[0].set_aspect('equal')
# ax[0].annotate("y={:.2f}x + {:.2f}; ".format(slope_pegs_aou, intercept_pegs_aou) + 
#                   r'$r=$' + "{:.2f}".format(rval_pegs_aou), 
#                   (1, 1), (0.02, 0.92), fontsize=8, xycoords='axes fraction')
# ax[0].annotate("A", (1, 1), (-0.32, 0.95), xycoords='axes fraction', fontsize=14, fontweight='bold')


ax.scatter(introgression_freq_aou_1kgp.freq_aou, introgression_freq_aou_1kgp.freq_1kgp, s=2)
ax.plot([0, 1], [0, 1], ls='--', color='black', transform=ax.transAxes)
ax.set_xlabel('AOU-EUR (10,000)')
ax.set_ylabel('1KGP-EUR (503)', labelpad=10)
ax.set_aspect('equal')
ax.annotate("y={:.2f}x + {:.2f}; ".format(slope_aou_1kgp, intercept_aou_1kgp) + 
                  r'$r=$' + "{:.2f}".format(rval_aou_1kgp), 
                  (1, 1), (0.02, 0.92), fontsize=8, xycoords='axes fraction')
# ax[1].annotate("B", (1, 1), (-0.175, 0.95), xycoords='axes fraction', fontsize=14, fontweight='bold')


# ax[2].scatter(introgression_freq_pegs_1kgp.freq_pegs, introgression_freq_pegs_1kgp.freq_1kgp, s=2)
# ax[2].plot([0, 1], [0, 1], ls='--', color='black', transform=ax[2].transAxes)
# ax[2].set_xlabel('PEGS-EUR (3,341)')
# ax[2].set_ylabel('1KGP-EUR (503)', labelpad=10)
# ax[2].set_aspect('equal')
# # ax[2].set_xlim([0, 0.6])
# # ax[2].set_ylim([0, 0.6])
# ax[2].annotate("y={:.2f}x + {:.2f}; ".format(slope_pegs_1kgp, intercept_pegs_1kgp) + 
#                   r'$r=$' + "{:.2f}".format(rval_pegs_1kgp), 
#                   (1, 1), (0.02, 0.92), fontsize=8, xycoords='axes fraction')
# ax[2].annotate("C", (1, 1), (-0.175, 0.95), xycoords='axes fraction', fontsize=14, fontweight='bold')



fig.savefig('visualizations/introgression_frequencies_1kgp_aou_50kb.pdf', dpi=600, bbox_inches='tight')
fig.savefig('visualizations/introgression_frequencies_1kgp_aou_50kb.png', dpi=600, bbox_inches='tight')





In [ ]:
from matplotlib.legend_handler import HandlerTuple
from scipy.stats import t


def get_min_sel_coeff(low, high):
    sel_coeff = np.stack([low, high])
    return np.where((sel_coeff[0, : ] < 0) & (sel_coeff[1, :] > 0), 0, np.abs(sel_coeff).min(axis=0))


def find_significant_segments(df, enriched=False, depleted=False, min_length=90000, windowsize=50000, 
                              stride=10000):
    if enriched:
        df = df[df.freq > df.expectations].copy()
    elif depleted:
        df = df[df.freq < df.expectations].copy()
    df.reset_index(drop=True, inplace=True)
    if windowsize > stride:
        min_windows = int((min_length - windowsize) / stride + 1)
    elif windowsize == stride:
        min_windows = int(min_length / stride)
    print(min_windows)
    idx_start = 0
    idx_end = 1
    differences = np.diff(df.start.values)
    to_keep = []
    for diff in differences:
        if diff == stride:
            idx_end += 1
        elif diff != stride and idx_end - idx_start >= min_windows:
            to_keep.append(np.arange(idx_start, idx_end + 1))
            idx_start = idx_end
            idx_end += 1
            
        else:
            idx_start = idx_end
            idx_end += 1
            
    if idx_end - idx_start >= min_windows:
        to_keep.append(np.arange(idx_start, idx_end))
    if len(to_keep) > 0 and df.shape[0] > 0:
        to_keep = np.concatenate(to_keep)
        # need to trait pvalues differently
        columns = np.where((df.columns.values != 'probabilities') & 
                           (df.columns.values != 'corrected'))[0][3:] + 1
        merged = BedTool.from_dataframe(df.iloc[to_keep]).merge(c=columns.tolist(),
                                                                o='mean',
                                                                header=True,
                                                                d=50000)
        df.drop(['probabilities', 'corrected'], axis=1, inplace=True)
        merged = merged.to_dataframe(names=df.columns)
    else:
        merged = pd.DataFrame(columns=df.columns)
    return merged


introgression_freq_filtered_fname = 'results50kb_wo_PEGS/ibdmix_Vindija33.19/ibdmix_results_masked_denisovan_combined_50kb_4.0LOD_afr_masked_coverage_per_individual_and_per_window50000_s_10000_pvalues.bed'
introgression_freq_filtered = pd.read_csv(introgression_freq_filtered_fname, sep='\t', header=0)
n_indv = ibdmix.loc[ibdmix['pop'] == 'AA', 'IID'].unique().shape[0]
introgression_freq_filtered.freq /= n_indv
introgression_freq_filtered.freq_updated /= n_indv
introgression_freq_filtered.expectations /= n_indv
introgression_freq_filtered.sel_coeff = np.nan_to_num(introgression_freq_filtered.sel_coeff, 
                                                      posinf=0.3, neginf=-0.3)
# introgression_freq_filtered['u'] = (np.abs(introgression_freq_filtered.freq_updated.values - 
#                                            introgression_freq_filtered.expectations.values) * 
#                                     introgression_freq_filtered.loc[:, ['freq_updated', 
#                                                                         'expectations']].min(axis=1))
# introgression_freq_filtered['min_sel_coeff'] = get_min_sel_coeff(introgression_freq_filtered.sel_coeff_low.values,
#                                                                  introgression_freq_filtered.sel_coeff_high.values)
# introgression_freq_filtered = BedTool.from_dataframe(introgression_freq_filtered).intersect(
#     BedTool.from_dataframe(pass_rec_filter), 
#     f=1).to_dataframe(names=introgression_freq_filtered.columns)
# introgression_freq_filtered.corrected = introgression_freq_filtered.probabilities * introgression_freq_filtered.shape[0]

fig = plt.figure(figsize=(8,4))
gs = fig.add_gridspec(46, 2, width_ratios=[0.5, 0.5], wspace=0.3)
ax0 = fig.add_subplot(gs[:, 0])
ax1 = fig.add_subplot(gs[3:43, 1])
# ax2 = fig.add_subplot(gs[1, :])
plt.subplots_adjust(wspace=0.3, hspace=0.4)


sd_drift = lambda x: np.sqrt(x * (1-x) * (1 - np.exp(-15 / 30000)))
lower_bound = t.ppf(0.025 / introgression_freq_filtered.shape[0], 30780, 
                   loc=introgression_freq_filtered.expectations.values,
                    scale=sd_drift(introgression_freq_filtered.expectations.values))
lower_bound = np.nan_to_num(np.where(lower_bound < 0, 0, lower_bound))
upper_bound = t.ppf(1 - 0.025 / introgression_freq_filtered.shape[0], 30780, 
                    loc=introgression_freq_filtered.expectations.values, 
                    scale=sd_drift(introgression_freq_filtered.expectations.values))
upper_bound = np.nan_to_num(upper_bound)

introgression_freq_filtered['drift_lower'] = lower_bound
introgression_freq_filtered['drift_upper'] = upper_bound

significant = introgression_freq_filtered[(introgression_freq_filtered.corrected < 0.05) &
                                          ((introgression_freq_filtered.freq_updated > 
                                            introgression_freq_filtered.drift_upper) | 
                                           (introgression_freq_filtered.freq_updated < 
                                            introgression_freq_filtered.drift_lower))]
non_significant = introgression_freq_filtered[(introgression_freq_filtered.corrected >= 0.05) |
                                              ((introgression_freq_filtered.freq_updated >= 
                                                introgression_freq_filtered.drift_lower) &
                                               (introgression_freq_filtered.freq_updated <= 
                                                introgression_freq_filtered.drift_upper))]
lower_bound = t.ppf(0.025 / introgression_freq_filtered.shape[0], 30780, 
                   loc=np.arange(0, 0.15, 0.0001), 
                    scale=sd_drift(np.arange(0, 0.15, 0.0001)))
lower_bound = np.nan_to_num(np.where(lower_bound < 0, 0, lower_bound))
upper_bound = t.ppf(1 - 0.025 / introgression_freq_filtered.shape[0], 30780, 
                    loc=np.arange(0, 0.15, 0.0001), 
                    scale=sd_drift(np.arange(0, 0.15, 0.0001)))
upper_bound = np.nan_to_num(upper_bound)

ax0.plot(np.arange(0, 0.15, 0.0001), upper_bound, ls='--', color='grey')
ax0.plot(np.arange(0, 0.15, 0.0001), lower_bound, ls='--', color='grey')

ax0.scatter(non_significant.expectations.values, non_significant.freq_updated.values, color='gray', 
           s=2, label='p' + r'$\geq$' + '{:.2e}'.format(0.05/introgression_freq_filtered.shape[0]))
ax0.scatter(significant.loc[significant.freq_updated > significant.expectations, 'expectations'].values, 
           significant.loc[significant.freq_updated > significant.expectations, 'freq'].values, s=2, c='blue', 
           label='p < {:.2e}'.format(0.05/introgression_freq_filtered.shape[0]), alpha=0.1)
ax0.scatter(significant.loc[significant.freq_updated < significant.expectations, 'expectations'].values, 
           significant.loc[significant.freq_updated < significant.expectations, 'freq'].values, s=2, c='red', 
           label='p < {:.2e}'.format(0.05/introgression_freq_filtered.shape[0]), alpha=0.1)

ax0.plot([0, 1], [0, 1], ls='--', c='black', label=f'y=x')


ax0.plot(np.arange(0, 0.15, 0.0001), upper_bound, ls='--', color='grey')
ax0.plot(np.arange(0, 0.15, 0.0001), lower_bound, ls='--', color='grey')


handles = [Line2D([0], [0], marker='o', markersize=2, color='gray', ls='',
                  label='P' + r'$\geq$' + '{:.2e}'.format(0.05/introgression_freq_filtered.shape[0])),
           (Line2D([0], [0], marker='o', markersize=2, color='red', ls=''),
            Line2D([0], [0], marker='o', markersize=2, color='blue', ls=''))]
ax0.legend(handles, [#'P' + r'$\geq$' + '{:.2e}'.format(0.05/introgression_freq_filtered.shape[0]),
                     #'P < {:.2e}'.format(0.05/introgression_freq_filtered.shape[0]),
                     'Not significant',
                     'Significantly depleted/enriched'], 
           bbox_to_anchor=(0.5, -.29), loc='center', ncol=3, fontsize=9, markerscale=2,
           handletextpad=0.4, columnspacing=1, handlelength=1.5, 
           handler_map={tuple: HandlerTuple(ndivide=None)})
ax0.set_xlabel("Exp. introgression frequency (50 kb)", fontsize=10)
ax0.set_ylabel("Obs. introgression frequency (50 kb)", fontsize=10)


ax0.set_aspect('equal')
ax0.set_xlim([0.0, 0.13])
ax0.set_ylim([0.0, 0.13])
inset = ax0.inset_axes([0.58, 0.03, 0.38, 0.38], xlim=(0, 0.04), ylim=(0, 0.04),
                         yticklabels=[], xticklabels=[], xticks=np.arange(0, 0.05, 0.01),
                         yticks=np.arange(0, 0.05, 0.01))
inset.scatter(non_significant.expectations.values, non_significant.freq_updated.values, color='gray', s=2)
inset.scatter(significant.loc[significant.freq_updated > significant.expectations, 'expectations'].values, 
              significant.loc[significant.freq_updated > significant.expectations, 'freq'].values, s=2, c='blue', 
              label='p < {:.2e}'.format(0.05/introgression_freq_filtered.shape[0]), alpha=0.1)
inset.scatter(significant.loc[significant.freq_updated < significant.expectations, 'expectations'].values, 
              significant.loc[significant.freq_updated < significant.expectations, 'freq'].values, s=2, c='red', 
              label='p < {:.2e}'.format(0.05/introgression_freq_filtered.shape[0]), alpha=0.1)
inset.plot([0, 1], [0, 1], ls='--', c='black')
inset.plot(np.arange(0, 0.15, 0.0001), upper_bound, ls='--', color='grey')
inset.plot(np.arange(0, 0.15, 0.0001), lower_bound, ls='--', color='grey')


ax0.indicate_inset_zoom(inset, edgecolor="black")
simulations = load_npz('simulations_freq_differences_50kb_windows.npz')
for i, diff in enumerate(simulations):
    ax1.hist(diff, bins=500,
             color='grey', histtype='step', weights = np.repeat(1 / diff.shape[0], diff.shape[0]),
             range=(-0.01, 0.01), alpha=0.4)
ax1.hist(introgression_freq_filtered.freq_updated - introgression_freq_filtered.expectations, 
         bins=500,
         color='red', histtype='step', weights = np.repeat(1 / introgression_freq_filtered.shape[0], 
                                                          introgression_freq_filtered.shape[0]),
         range=(-0.01, 0.01), label='Admixed individuals')
ax1.axvline(0.0, ls='--', color='black')
ax1.set_ylabel("Density", fontsize=10)
ax1.set_xlabel('Obs. - Exp. introgression frequency (50 kb)', fontsize=10)
handles = [Line2D([0], [0], ls='-', color='grey', label='Simulations'),
           Line2D([0], [0], ls='-', color='red', label='AOU-Admixed')]
ax1.legend(handles=handles, bbox_to_anchor=(0.5, -.29), loc='center', ncol=2, markerscale=2,
           handletextpad=0.4, columnspacing=1, handlelength=1.5, fontsize=9)

ax1.set_yscale('log')
ax0.annotate("A", (1, 1), (-0.35, 0.9), fontsize=16, fontweight='bold', xycoords='axes fraction')
ax1.annotate("B", (1, 1), (-0.25, 0.9), fontsize=16, fontweight='bold', xycoords='axes fraction')
fig.savefig("visualizations/nea_enrichment_50kb.pdf", bbox_inches='tight', dpi=600)
fig.savefig("visualizations/nea_enrichment_50kb.png", bbox_inches='tight', dpi=600)

In [ ]:
BedTool.from_dataframe(significant[significant.freq_updated > significant.expectations]).merge(d=50000).to_dataframe().set_index('chrom').loc['chr6']

In [ ]:
simulated_introgression_freq_filtered = pd.read_csv('simulated_introgression_freq_filtered.bed', 
                                                    sep='\t', header=0)

import seaborn as sns
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
import scipy.ndimage as ndimage
from matplotlib.patches import Ellipse

def get_peaks(residuals):
    residuals = ndimage.gaussian_filter(residuals, sigma=2, radius=6)
    residuals = np.where(residuals < 1e-6, 0, residuals)
    distance = ndimage.distance_transform_edt(residuals)
    coords = peak_local_max(distance, footprint=np.ones((6, 6)), threshold_abs=1e-6)
    mask = np.zeros(residuals.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = ndimage.label(mask)
    labels = watershed(-distance, markers, mask=residuals)
    return labels

def extract_bins_in_watershed(df, bins, peaks):
    significant = []
    bin_size = np.diff(bins)[0]
    for label in np.unique(peaks):
        if label == 0:
            continue
        bins_x = bins[np.where(peaks == label)[1]]
        bins_y = bins[np.where(peaks == label)[0]]
        for x, y in zip(bins_x, bins_y):
            significant.append(df[(df.expectations >= x) &
                                  (df.expectations < x + bin_size) &
                                  (df.freq_updated >= y) &
                                  (df.freq_updated < y + bin_size)])
    return pd.concat(significant).sort_values(['chrom', 'start'])

def match_residuals_to_windows(df, bins, residuals):
    residuals = ndimage.gaussian_filter(residuals, sigma=2, radius=6)
    residuals = np.where(residuals < 1e-6, 0, residuals)
    distance = ndimage.distance_transform_edt(residuals)
    distance = distance[np.digitize(df.freq_updated, bins[:-1], right=True) - 1, 
                        np.digitize(df.expectations, bins[:-1], right=True) - 1]
    return distance

fig = plt.figure(figsize=(8, 8))
gs = fig.add_gridspec(2, 14, wspace=0.4, hspace=0.3)
ax0 = fig.add_subplot(gs[0, 0:6])
ax1 = fig.add_subplot(gs[0, 8:])
ax2 = fig.add_subplot(gs[1, 4:10])
# ax3 = fig.add_subplot(gs[1, :])
plt.subplots_adjust(wspace=0.75)


nbins = 50
bins = np.linspace(0, 0.1, nbins)
# get heatmaps of expectations vs observations
heatmap, _, _ = np.histogram2d(introgression_freq_filtered.expectations, 
                               introgression_freq_filtered.freq_updated, bins=bins)

heatmap_simulated, _, _ = np.histogram2d(simulated_introgression_freq_filtered.expectations, 
                                         simulated_introgression_freq_filtered.freq_updated, 
                                         bins=bins)
#plot heat maps
divider = make_axes_locatable(ax0)
cax0 = divider.append_axes('right', size='5%', pad=0.05)
im0 = ax0.imshow(heatmap.T / introgression_freq_filtered.shape[0], 
                norm=LogNorm(),
                origin='lower', cmap='plasma_r')
cbar0 = fig.colorbar(im0, cax=cax0, orientation='vertical')
cbar0.set_label("Density", fontsize=9)
# for t in cbar0.ax.get_yticklabels():
#      t.set_fontsize(8)

divider = make_axes_locatable(ax1)
cax1 = divider.append_axes('right', size='5%', pad=0.05)
im1 = ax1.imshow(heatmap_simulated.T / simulated_introgression_freq_filtered.shape[0], 
                 norm=LogNorm(), origin='lower', cmap='plasma_r')
cbar1 = fig.colorbar(im1, cax=cax1, orientation='vertical')
cbar1.set_label("Density", fontsize=9)
# for t in cbar1.ax.get_yticklabels():
#      t.set_fontsize(8)


# calculate residuals
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes('right', size='5%', pad=0.05)
residuals = ((heatmap.T / introgression_freq_filtered.shape[0]) -
             (heatmap_simulated.T / simulated_introgression_freq_filtered.shape[0]))
# find watersheds
peaks = get_peaks(residuals)
introgression_freq_filtered['residuals'] = match_residuals_to_windows(introgression_freq_filtered, bins, 
                                                                      residuals)
# residuals = np.where(residuals < 0, 0, residuals)
# residuals = np.interp(residuals, (residuals.min(), residuals.max()), (0, 10))
# residuals = np.nan_to_num(residuals, 0)
im2 = ax2.imshow(np.where(residuals < 1e-6, 0, residuals), norm=LogNorm(), 
                 origin='lower', cmap='plasma_r')
cbar2 = fig.colorbar(im2, cax=cax2, orientation='vertical')
cbar2.set_label("Residuals", fontsize=9)
# for t in cbar2.ax.get_yticklabels():
#      t.set_fontsize(9)
        
### THIS PART IS NOT GENERAL FOR OTHER CASES, I'M JUST MERGING PEAKS HERE TO GET A CLEANER FIGURE
peaks = np.where((peaks > 0) & (peaks <= 3), 1, peaks)
peaks = np.where((peaks > 3), 2, peaks)

for label in np.unique(peaks):
    if label != 0:
        ellipse = Ellipse(((np.where(peaks == label)[1].max() + np.where(peaks == label)[1].min()) / 2,
                           (np.where(peaks == label)[0].max() + np.where(peaks == label)[0].min()) /2), 
                          width=(np.where(peaks == label)[1].max() + np.where(peaks == label)[1].min()) / 2, 
                          height=(np.where(peaks == label)[0].max() + np.where(peaks == label)[0].min()) /2,
                          fc='None', edgecolor='black')
        ax2.add_patch(ellipse)

ax0.plot([0, 1], [0, 1], ls='--', c='black', transform=ax0.transAxes)
ax1.plot([0, 1], [0, 1], ls='--', c='black', transform=ax1.transAxes)
ax2.plot([0, 1], [0, 1], ls='--', c='black', transform=ax2.transAxes)


ax0.set_xlim([0, nbins])
ax0.set_ylim([0, nbins])
ax0.set_xticks(np.linspace(0, nbins, 6))
ax0.set_yticks(np.linspace(0, nbins, 6))
ax0.set_xticklabels(['{:.2f}'.format(label) for label in np.linspace(0, 0.1, 6)],
                   rotation=60)
ax0.set_yticklabels(['{:.2f}'.format(label) for label in np.linspace(0, 0.1, 6)])
ax0.set_aspect('equal')
ax1.set_xticks(np.linspace(0, nbins, 6))
ax1.set_yticks(np.linspace(0, nbins, 6))
ax1.set_xticklabels(['{:.2f}'.format(label) for label in np.linspace(0, 0.1, 6)],
                   rotation=60)
ax1.set_yticklabels([])
ax2.set_xticks(np.linspace(0, nbins, 6))
ax2.set_yticks(np.linspace(0, nbins, 6))
ax2.set_xticklabels(['{:.2f}'.format(label) for label in np.linspace(0, 0.1, 6)], 
                    rotation=60)
ax2.set_yticklabels([])

selected = extract_bins_in_watershed(introgression_freq_filtered, bins, peaks)
selected.sort_values(['chrom', 'start'], inplace=True)
depleted_segments = find_significant_segments(selected, depleted=True, min_length=50000)
enriched_segments = find_significant_segments(selected, enriched=True, min_length=50000)

# introgression_freq_filtered.residuals += np.abs(introgression_freq_filtered.residuals.min())
# introgression_freq_filtered.residuals /= introgression_freq_filtered.residuals.max()
enriched_windows = BedTool.from_dataframe(introgression_freq_filtered).intersect(
    BedTool.from_dataframe(enriched_segments), f=1).to_dataframe(names=introgression_freq_filtered.columns)
depleted_windows = BedTool.from_dataframe(introgression_freq_filtered).intersect(
    BedTool.from_dataframe(depleted_segments), f=1).to_dataframe(names=introgression_freq_filtered.columns)
ax0.set_xlabel("Exp. introgression\nfrequency (50 kb window)")
ax1.set_xlabel("Exp. introgression\nfrequency (50 kb window)")
ax2.set_xlabel("Exp. introgression\nfrequency (50 kb window)")
ax0.set_ylabel("Obs. introgression\nfrequency (50 kb window)")
ax2.set_ylabel("Obs. introgression\nfrequency (50 kb window)")


ax0.annotate("A", (1, 1), (-0.3, 0.95), fontsize=14, fontweight='bold', xycoords='axes fraction')
ax1.annotate("B", (1, 1), (-0.15, 0.95), fontsize=14, fontweight='bold', xycoords='axes fraction')
ax2.annotate("C", (1, 1), (-0.2, 0.95), fontsize=14, fontweight='bold', xycoords='axes fraction')
# get windows in watershed
fig.savefig('visualizations/figure4.pdf', bbox_inches='tight', dpi=600)
fig.savefig('visualizations/figure4.png', bbox_inches='tight', dpi=600)



In [ ]:
residuals = ((heatmap.T / introgression_freq_filtered.shape[0]) -
             (heatmap_simulated.T / simulated_introgression_freq_filtered.shape[0]))

residuals = ndimage.gaussian_filter(residuals, sigma=2, radius=6)
residuals = np.where(residuals < 1e-6, 0, residuals)
distance = ndimage.distance_transform_edt(residuals)
coords = peak_local_max(distance, footprint=np.ones((6, 6)), threshold_abs=1e-6)
mask = np.zeros(residuals.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndimage.label(mask)
labels = watershed(-distance, markers, mask=residuals)


fig, ax = plt.subplots()
ax.imshow(np.where((labels > 0) & (labels <= 3), 1, labels), cmap=plt.cm.nipy_spectral)
ax.invert_yaxis()

In [ ]:
selected = extract_bins_in_watershed(introgression_freq_filtered, bins, peaks)
depleted_segments = find_significant_segments(selected.sort_values(['chrom', 'start']),
                                              depleted=True, min_length=50000)
enriched_segments = find_significant_segments(selected.sort_values(['chrom', 'start']), 
                                              enriched=True, min_length=50000)
print(enriched_segments.iloc[:, :3])
print(depleted_segments.iloc[:,:3])

In [ ]:
depleted_segments.chrom = [int(chrom.replace('chr', '')) for chrom in depleted_segments.chrom]
enriched_segments.chrom = [int(chrom.replace('chr', '')) for chrom in enriched_segments.chrom]
outlier_regions = pd.concat([depleted_segments.sort_values('chrom'), 
                             enriched_segments.sort_values('chrom')])

In [ ]:
e

In [ ]:
def read_gtf(file_path):
    # Define column names as GTF does not have a header
    column_names = [
        'seqname', 'source', 'feature', 'start', 'end',
        'score', 'strand', 'frame', 'attribute'
    ]
    
    # Read the GTF file using pandas
    df = pd.read_csv(
        file_path,
        sep='\t',
        comment='#',
        names=column_names,
        dtype={
            'seqname': str, 'source': str, 'feature': str,
            'start': int, 'end': int, 'score': str,
            'strand': str, 'frame': str, 'attribute': str
        }
    )
    
    return df

# Example usage
gtf_file_path = 'data/reference/gencode.v46.annotation.gtf.gz'
gtf_df = read_gtf(gtf_file_path)
gtf_df['gene_type'] = [attribute.split('gene_type "')[1].split('";')[0] for attribute in gtf_df.attribute]
gtf_df['gene_name'] = [attribute.split('gene_name "')[1].split('";')[0] for attribute in gtf_df.attribute]
gtf_df['gene_id'] = [attribute.split('gene_id "')[1].split('";')[0] for attribute in gtf_df.attribute]

gtf_df.drop('attribute', axis=1, inplace=True)
gtf_df.reset_index(inplace=True, drop=True)


In [ ]:
from matplotlib.legend_handler import HandlerTuple
import string


fig = plt.figure(figsize=(8, 16))

gs_outer = fig.add_gridspec(189, 1, hspace=0.5)
n_rows = 0
introgression_freq = pd.read_csv('results50kb_wo_PEGS/ibdmix_Vindija33.19/ibdmix_results_masked_denisovan_combined_50kb_4.0LOD_afr_masked_coverage_per_individual_and_per_window50000_s_10000_expectations.bed',
                                 sep='\t', header=0)
plt.subplots_adjust(hspace=(0.6))
alphabet = string.ascii_uppercase
for i, (chrom, start, end, 
        freq, expectation) in enumerate(outlier_regions.loc[:, ['chrom', 'start', 'end', 
                                                             'freq_updated', 'expectations']].values):
    c_df = introgression_freq[(introgression_freq.chrom == f'chr{int(chrom)}') &
                              (introgression_freq.start >= start - 500000) &
                              (introgression_freq.end <= end + 500000)]
    x_coords = (c_df.end + c_df.start).values / 2 / 1e6
    expected = c_df.expectations.values / n_indv
    observed = c_df.freq_updated.values / n_indv
    genes = gtf_df[(gtf_df.seqname == f'chr{int(chrom)}') &
                   ((gtf_df.end > start - 500000) &
                   (gtf_df.start < end + 500000))]
    genes = genes[genes.gene_type == 'protein_coding']
    
    gs = gridspec.GridSpecFromSubplotSpec(7 + genes.gene_name.unique().shape[0] * 2 + 1, 1, hspace=1,
                                          subplot_spec = gs_outer[n_rows: n_rows + 7 +
                                                                  genes.gene_name.unique().shape[0] * 2 + 1])
    n_rows += 7 + genes.gene_name.unique().shape[0] * 2
    ax = fig.add_subplot(gs[:5])
#     ax1 = fig.add_subplot(gs[5:7])
    
    if freq > expectation:
        color='blue'
    else:
        color='red'
    if x_coords[0] > (start - 500000) / 1e6:
        x_coords = np.concatenate([[(start - 500000) / 1e6], x_coords])
        expected = np.concatenate([[0], expected])
        ax2.set_yticks([])
        observed = np.concatenate([[0], observed])
    if x_coords[-1] < (end + 500000) / 1e6:
        x_coords = np.concatenate([x_coords, [(end + 500000) / 1e6]])
        expected = np.concatenate([expected, [0]])
        observed = np.concatenate([observed, [0]])
    
    ax.annotate(alphabet[i], (1, 1), (-0.05, 1.1), fontsize=14, fontweight='bold', xycoords='axes fraction')
    ax.plot(x_coords, observed, ls='-',
               color='black', label='Observed')
    ax.plot(x_coords, expected, ls='--',
           color='grey', label="Expected")
#     ax1 = fig.add_subplot(gs[5: 7], sharex=ax)
#     ax1.bar(start / 1e6, 1,
#               width = (end - start) / 1e6, color=color, align='edge')
    for i, gene in enumerate(genes.gene_name.unique()):    
        ax2 = fig.add_subplot(gs[7 + i * 2 : 7 + i * 2 + 2])
        gstart = genes.loc[(genes.gene_name == gene) &
                            (genes.feature == 'gene'), 'start'].values[0]
        gend = genes.loc[(genes.gene_name == gene) &
                            (genes.feature == 'gene'), 'end'].values[0]
        if gstart < start - 500000:
            gstart = start - 500000
        if gend > end + 500000:
            gend = end + 500000
        gstrand = genes.loc[(genes.gene_name == gene) &
                            (genes.feature == 'gene'), 'strand'].values[0]
        

        for estart, eend, estrand in genes.loc[(genes.gene_name == gene) & 
                                               (genes.feature == 'exon'), ['start', 'end', 'strand']].values:
            if estart < start - 500000:
                estart = start - 500000
            if eend > end + 500000:
                eend = end + 500000
#             if estart == gstart:
#                 estart += 50000
#             if end == gend + 500000:
#                 eend -= 50000
            if gstrand == '+':
#                 ax2.annotate("", xy=(estart / 1e6, 0), xytext=(eend / 1e6, 0), color='grey', 
#                          arrowprops=dict(arrowstyle="<-"))
                ax2.bar(estart / 1e6, height=0.4,
                      width = (eend - estart) / 1e6, color='black',
                      bottom=0.8, align='edge')
            else:
#                 ax2.annotate("", xy=(estart / 1e6, 0), xytext=(eend / 1e6, 0), color='grey', 
#                          arrowprops=dict(arrowstyle="->"))
                ax2.bar(eend / 1e6, height=0.4,
                      width = (estart - eend) / 1e6, color='black',
                      bottom=0.8, align='edge')
        if gstrand == '+':
            ax2.annotate("", xy=(gstart / 1e6, 1), xytext=(gend / 1e6 + 0.01, 1), color='grey', 
                         arrowprops=dict(arrowstyle="<-", color='black'))
        else:
            ax2.annotate("", xy=(gstart / 1e6 - 0.01, 1), xytext=(gend / 1e6, 1), color='grey', 
                         arrowprops=dict(arrowstyle="->", color='black'))
        ax2.annotate(gene, (1, 1), (1, 0.5), xycoords='axes fraction', fontsize=9, ha='left', va='center')
        ax2.spines['top'].set_visible(False)
        ax2.spines['right'].set_visible(False)
        ax2.spines['left'].set_visible(False)
        ax2.spines['bottom'].set_visible(False)
        ax2.set_yticks([])
        ax2.set_xticks([])
        
        ax2.set_xlim([(start - 500000)  / 1e6 , (end + 500000) / 1e6])    
                
    ax.set_xlim([(start - 500000)  / 1e6 , (end + 500000) / 1e6])    
    ax.set_title(f'chr{int(chrom)}:{int(start - 500000):,}-{int(end + 500000):,}', fontsize=10,
                 pad=10)
    ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right") 
    ax.tick_params(axis='y', labelsize=9)
    ax.set_xticks([])
#     ax1.set_xlim([(start - 500000)  / 1e6 , (end + 500000) / 1e6])    
#     ax1.spines['top'].set_visible(False)
#     ax1.spines['right'].set_visible(False)
#     ax1.spines['left'].set_visible(False)
#     ax1.spines['bottom'].set_visible(False)
#     ax1.set_yticks([])
#     ax1.set_xticks([])
    

    ax2 = fig.add_subplot(gs[-1])
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.set_yticks([])
    ax2.spines['left'].set_visible(False)
    ax2.set_xlim([(start - 500000)  / 1e6 , (end + 500000) / 1e6])
    ax2.set_yticks([])
    ax2.set_xticks(np.arange((start - 500000) / 1e6, (end + 500000) /1e6, 200000 / 1e6))
    n_rows += 13
ax2.set_xlabel('Chromosomal position (Mb))')
handles = [Line2D([0], [0], ls='--', color='grey'),
           Line2D([0], [0], ls='-', color='black')]
ax2.legend(handles, ['Expected introgression frequency',
                     'Observed introgression frequency'], 
           bbox_to_anchor=(0.5, -25), loc='center', ncol=2, fontsize=9, markerscale=2,
           handletextpad=0.4, columnspacing=1, handlelength=1.5, 
           handler_map={tuple: HandlerTuple(ndivide=None)})

ax = fig.add_subplot(gs_outer[n_rows:])
ax.remove()
#     ax2.spines['bottom'].set_visible(True)
    
# #     ax[i].set_xticks((c_df.end + c_df.start)[::10] / 2 / 1e6)
# #     ax[i].set_xticklabels((c_df.end + c_df.start)[::10] / 2 / 1e6)
# ax[i].set_xlabel('Chromosomal position (Mb)')
# ax[i].legend(bbox_to_anchor=(0.5, -.5), loc='upper center', ncol=2)
fig.savefig('visualizations/figure5.pdf', bbox_inches='tight', dpi=600)
fig.savefig('visualizations/figure5.png', bbox_inches='tight', dpi=600)

### Introgression landscape

In [ ]:
%matplotlib inline
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, Patch

deserts = pd.read_csv('results50kb_wo_PEGS/ibdmix_Vindija33.19/AMR_novel_introgression_deserts.bed', sep='\t',
                      names=['chrom', 'start', 'end'])
deserts_eur = pd.read_csv('data/known_introgression_deserts_hg38.bed', sep='\t', 
                          names=['chrom', 'start', 'end', 'reference','fragment'])
deserts = BedTool.from_dataframe(deserts).intersect(BedTool.from_dataframe(deserts_eur), 
                                                    v=True).to_dataframe()
deserts_eur = BedTool.from_dataframe(deserts_eur).merge().to_dataframe()

# merged_introgression_freq = BedTool.from_dataframe(introgression_freq).merge(c=[4, 6, 7], o='mean').to_dataframe()
# merged_introgression_freq = merged_introgression_freq.rename(columns={'name': 'AMR_freq', 'score': 'EUR_freq',
#                                                                      'strand': "EAS_freq"})
# introgression_freq.chrom = [int(chrom.replace('chr', '')) for chrom in introgression_freq.chrom]
chromosome_arms = pd.read_csv('data/reference/hg38_chromosome_arms.bed', sep='\t',
                              names=['chrom', 'start', 'end', 'arm'])
max_chrom_size = chromosome_arms.end.max() / 1e6
chromosome_arms = chromosome_arms[chromosome_arms.arm == 'p']
fig, ax = plt.subplots(22, 1, figsize=(6, 9), sharex=True)
max_freq = 1
for chrom in tqdm(range(1, 23)):
    desert_patches = []
    known_deserts_patches = []
        
    ax[chrom - 1] = plot_haplotype_frequencies(ibdmix_masked[ibdmix_masked.super_pop == 'AMR'].drop_duplicates(['chrom', 'start', 'end']),
                                               'freq_updated', f'chr{chrom}', ax[chrom - 1], 1, 
                                               'darkmagenta')
    if deserts[deserts.chrom == f'chr{chrom}'].shape[0] > 0:
        for start, end in deserts.loc[deserts.chrom == f'chr{chrom}', ['start', 'end']].values:
            desert_patches.append(Rectangle((start / 1e6, 0.0), (end - start) / 1e6, max_freq))
            
        pc = PatchCollection(desert_patches, edgecolor='red', facecolor='red', alpha=0.2)
        ax[chrom - 1].add_collection(pc)
    if deserts_eur[deserts_eur.chrom == f'chr{chrom}'].shape[0] > 0:
        for start, end in deserts_eur.loc[deserts_eur.chrom == f'chr{chrom}', ['start', 'end']].values:
            known_deserts_patches.append(Rectangle((start / 1e6, 0.0), (end - start) / 1e6, max_freq))
            
        pc = PatchCollection(known_deserts_patches, edgecolor='orange', facecolor='orange', 
                             alpha=0.2)
        ax[chrom - 1].add_collection(pc)
    ax[chrom - 1].scatter(chromosome_arms.loc[chromosome_arms.chrom == chrom, 'end'].values[0] / 1e6, 
                          max_freq / 2, s=15, c='black')
    ax[chrom - 1].set_yticks([max_freq / 2])
    ax[chrom - 1].set_yticklabels([chrom])
    ax[chrom - 1].set_ylim([-0.05, max_freq])
    for spine in ax[chrom-1].spines.values():
        spine.set_visible(False)
    if chrom != 22:
        ax[chrom - 1].tick_params(axis='both', bottom=False, left=False)
    else:
        ax[chrom - 1].tick_params(axis='both', bottom=True, left=False)
ax[chrom - 1].set_xlim([0, max_chrom_size])
ax[chrom - 1].set_xlabel('Coordinates in Mb')
legend = [Patch(facecolor='darkmagenta', edgecolor='darkmagenta', alpha=0.3, 
                label='Neanderthal segments'),
#           Line2D([0], [0], ls='-', color='lightblue', alpha=.2, label="Introgression freq. EUR"),
#           Line2D([0], [0], ls='-', color='blue', alpha=.1, label="Introgression freq. EAS"),
          Patch(facecolor='red', edgecolor='red', alpha=0.3, 
                label='Novel introgression desert-like region'),
          Patch(facecolor='orange', edgecolor='orange', alpha=0.3, 
                label='Vernot et al. 2016 + Chen et al. 2020')]
ax[chrom - 1].legend(handles=legend, ncol=1, bbox_to_anchor=(0.6, 4),
                     loc='upper center', fontsize='small')
ax[0].annotate("A", (1, 1), (-0.15, 0.3), fontsize=16, fontweight='bold', xycoords='axes fraction')
fig.supylabel('Chromosomes', x=0.04)
fig.savefig("visualizations/introgression_landscape.pdf", bbox_inches='tight', dpi=600)
fig.savefig("visualizations/introgression_landscape.png", bbox_inches='tight', dpi=600)

### Plot statistics for introgression deserts

In [ ]:
df_introgressed = []
df_ils = []
df_non_introgressed = []
dfs = []
# chromosomes = np.arange(1, 23)
for chrom in tqdm.tqdm(chromosomes):
    df = pd.read_csv(f'../ancestral_recombination_graphs/chr{chrom}_tmrca_summarized_hg38.tab', sep='\t', 
                     header=0)
    df.dropna(inplace=True)
    columns = df.columns.values.tolist()
    columns.insert(2, 'end')
    df['end'] = df.POS + 1
    df = df.loc[:, columns]
    dfs.append(df)
    # derived allele found in present-day Europeans but not in Africans
    and in at least one Neanderthal but Denisovan

df = pd.concat(dfs)
df = BedTool.from_dataframe(df).intersect('data/masks/cpg.bed', v=True).to_dataframe(names=df.columns)

df_introgressed = df[(df.AFR_1KGP == 0) & (df.EUR_1KGP > 0) &
                     ((df.Nea_Altai > 0) | (df['Nea_Vindija33.19'] > 0) | (df.Nea_Chagyrskaya > 0)) & 
                     (df.Denisova == 0)]
# derived allele found in present-day Europeans but not in Africans
# and in at least one Neanderthal and Denisovan --> likely ILS
df_ils = df[(df.AFR_1KGP == 0) & (df.EUR_1KGP == 0) &
            ((df.Nea_Altai > 0) | (df['Nea_Vindija33.19'] > 0) | (df.Nea_Chagyrskaya > 0)) & 
            (df.Denisova > 0)]
df_non_introgressed = df[(df.AFR_1KGP == 0) & (df.EUR_1KGP > 0) & 
                         (df.Nea_Altai == 0) & (df['Nea_Vindija33.19'] == 0) & 
                         (df.Nea_Chagyrskaya == 0) & (df.Denisova == 0)]

In [ ]:
fig = plt.figure(figsize=(3, 9))

gs = fig.add_gridspec(7, 1, height_ratios=[10, 2, 10, 2, 10, 1, 1])
ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[2])
ax2 = fig.add_subplot(gs[4])
ax3 = fig.add_subplot(gs[6])


deserts = pd.read_csv('results50kb_wo_PEGS/ibdmix_Vindija33.19/AMR_novel_introgression_deserts.bed', sep='\t',
                      names=['chrom', 'start', 'end'])
deserts_eur = pd.read_csv('data/known_introgression_deserts_hg38.bed', sep='\t', 
                          names=['chrom', 'start', 'end', 'reference','fragment'])
deserts = BedTool.from_dataframe(deserts).intersect(BedTool.from_dataframe(deserts_eur), 
                                                    v=True).to_dataframe()
deserts_eur = BedTool.from_dataframe(deserts_eur).merge().to_dataframe()
bstatistic = pd.read_csv('data/reference/bstat_hg38.bed.gz', sep='\t', 
                         names=['chrom_r', 'start_r', 'end_r', "B"])
columns = ['chrom', 'start', 'end', 'chrom_r', 'start_r', 'end_r', "B", 'overlap']
deserts_b = BedTool.from_dataframe(deserts).intersect(
    BedTool.from_dataframe(bstatistic), wao=True).to_dataframe(names=columns)
deserts_eur_b = BedTool.from_dataframe(deserts_eur).intersect(
    BedTool.from_dataframe(bstatistic), wao=True).to_dataframe(names=columns)

ax0.step(np.sort(bstatistic.B.values)  / 1000, 
        np.cumsum((bstatistic.end_r.values - bstatistic.start_r.values)[np.argsort(bstatistic.B.values)] / 
        (bstatistic.end_r - bstatistic.start_r).sum()), ls='--', color='black', label='Genome-wide background' )
ax0.step(np.sort(deserts_b.B.values) / 1000, np.cumsum(deserts_b.overlap.values[np.argsort(deserts_b.B.values)] / 
                                             deserts_b.overlap.sum()), 
        ls='-', color='red', label='Novel desert-like regions')
ax0.step(np.sort(deserts_eur_b.B.values) / 1000, 
        np.cumsum(deserts_eur_b.overlap.values[np.argsort(deserts_eur_b.B.values)] / 
                  deserts_eur_b.overlap.sum()), 
        ls='-', color='orange', label='Known deserts')
ax0.set_xlabel("B-statistic")
ax0.set_ylabel('ECDF')
ax0.annotate("B", (1, 1), (-0.275, 0.9), fontsize=16, fontweight='bold', xycoords='axes fraction')


deserts = pd.read_csv('results50kb_wo_PEGS/ibdmix_Vindija33.19/AMR_novel_introgression_deserts.bed', sep='\t',
                      names=['chrom', 'start', 'end'])
deserts_eur = pd.read_csv('data/known_introgression_deserts_hg38.bed', sep='\t', 
                          names=['chrom', 'start', 'end', 'reference','fragment'])
deserts = BedTool.from_dataframe(deserts).intersect(BedTool.from_dataframe(deserts_eur), 
                                                    v=True).to_dataframe()
deserts_eur = BedTool.from_dataframe(deserts_eur).merge().to_dataframe()

desert_vals = []
for chrom, start, end in deserts.loc[:, ['chrom', 'start', 'end']].values:
    desert_vals.append(df_introgressed.loc[(df_introgressed.CHROM == chrom) & 
                                           (df_introgressed.POS >= start) & 
                                           (df_introgressed.POS < end), 'TMRCA_kya'].values)
desert_vals = np.concatenate(desert_vals)

desert_eur_vals = []
for chrom, start, end in deserts_eur.loc[:, ['chrom', 'start', 'end']].values:
    desert_eur_vals.append(df_introgressed.loc[(df_introgressed.CHROM == chrom) & 
                                               (df_introgressed.POS >= start) & 
                                               (df_introgressed.POS < end), 'TMRCA_kya'].values)
desert_eur_vals = np.concatenate(desert_eur_vals)


ax1.step(np.sort(df_introgressed.TMRCA_kya.values),
        np.cumsum(np.repeat(1 / df_introgressed.shape[0], df_introgressed.shape[0])), ls='--', color='black',
        label='Genome-wide background')
ax1.step(np.sort(desert_vals),
        np.cumsum(np.repeat(1 / len(desert_vals), len(desert_vals))), ls='-', color='red', 
        label='Novel desert-like regions')
ax1.step(np.sort(desert_eur_vals),
        np.cumsum(np.repeat(1 / len(desert_eur_vals), len(desert_eur_vals))), ls='-', color='orange', 
        label='Known deserts')
# ax.legend(bbox_to_anchor=(0.5, -.14), loc='upper center', ncol=3)
ax1.set_xlabel("TMRCA in kya")
ax1.set_ylabel('ECDF')
ax1.set_xlim([0, 1000])
ax1.annotate("C", (1, 1), (-0.275, 0.9), fontsize=16, fontweight='bold', xycoords='axes fraction')




deserts = pd.read_csv('results50kb_wo_PEGS/ibdmix_Vindija33.19/AMR_novel_introgression_deserts.bed', sep='\t',
                      names=['chrom', 'start', 'end'])
deserts_eur = pd.read_csv('data/known_introgression_deserts_hg38.bed', sep='\t', 
                          names=['chrom', 'start', 'end', 'reference','fragment'])
deserts = BedTool.from_dataframe(deserts).intersect(BedTool.from_dataframe(deserts_eur), 
                                                    v=True).to_dataframe()
deserts_eur = BedTool.from_dataframe(deserts_eur).merge().to_dataframe()
phastcons = pd.read_csv('data/reference/phastCons100way.bed.gz', sep='\t', 
                         names=['chrom_r', 'start_r', 'end_r', "lowerLimit", "upperLimit",  "phastCons"])
columns = ['chrom', 'start', 'end', 'chrom_r', 'start_r', 'end_r', "lowerLimit", 
           "upperLimit",  "phastCons", 'overlap']
deserts_pc = BedTool.from_dataframe(deserts).intersect(
    BedTool.from_dataframe(phastcons), wao=True).to_dataframe(names=columns)
deserts_eur_pc = BedTool.from_dataframe(deserts_eur).intersect(
    BedTool.from_dataframe(phastcons), wao=True).to_dataframe(names=columns)

ax2.step(np.sort(phastcons.phastCons.values), 
        np.cumsum((phastcons.end_r.values - phastcons.start_r.values)[np.argsort(phastcons.phastCons.values)] / 
        (phastcons.end_r - phastcons.start_r).sum()), ls='--', color='black', 
           label='Genome-wide background' )
ax2.step(np.sort(deserts_pc.phastCons.values), 
           np.cumsum(deserts_pc.overlap.values[np.argsort(deserts_pc.phastCons.values)] / 
                     deserts_pc.overlap.sum()), 
        ls='-', color='red', label='Novel desert-like regions')
ax2.step(np.sort(deserts_eur_pc.phastCons.values), 
        np.cumsum(deserts_eur_pc.overlap.values[np.argsort(deserts_eur_pc.phastCons.values)] / 
                  deserts_eur_pc.overlap.sum()), 
        ls='-', color='orange', label='Known deserts')
ax2.set_xlabel("phastCons score")
ax2.set_ylabel('ECDF')
ax2.set_xscale('log')

ax2.annotate("D", (1, 1), (-0.275, 0.9), fontsize=16, fontweight='bold', xycoords='axes fraction')


handles = [Line2D([0] , [0], color='black', ls='--', label='Genomic background'),
           Line2D([0] , [0], color='red', ls='--', label='Novel desert-like regions'),
           Line2D([0] , [0], color='red', ls='--', label='Known deserts')]
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.set_xticks([])
ax3.set_yticks([])

ax3.legend(handles=handles, bbox_to_anchor=(0.4, 1),loc='upper center', ncol=2, handlelength=1, 
             handletextpad=0.4, labelspacing=0.3, columnspacing=1)


fig.savefig('visualizations/desert_like_regions_stats.pdf', bbox_inches='tight', dpi=600)
fig.savefig('visualizations/desert_like_regions_stats.png', bbox_inches='tight', dpi=600)

